<a href="https://colab.research.google.com/github/AlexWalcher/optimizationOfHintGeneration/blob/Test/optimizationOfHintGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##How to use:
1. Run **All imports**
2. Run **Load File**
3. Run **New prediction method for years** or any of the **Old predictions methods**

# **All imports**

In [1]:
%%capture
# This cell will not display any output

%cd /content/
%rm -r optimizationOfHintGeneration
!git clone https://github.com/AlexWalcher/optimizationOfHintGeneration.git
%cd /content/

In [2]:
%%capture
# This cell will not display any output

!pip install selenium
!apt-get update 
!apt-get install firefox
!apt install firefox-geckodriver

!pip install sparqlwrapper
!pip install pageviewapi
!pip install sentence-transformers
!pip install wikipedia

In [3]:
%%capture
# This cell will not display any output

#new imports
import random
import spacy
import time
import requests
import re
import difflib
import pprint
import itertools
import wikipedia
# import wikipediaapi

import pandas as pd
pd.set_option('display.max_colwidth',1000)

from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

import bs4
from bs4 import BeautifulSoup, NavigableString, Tag

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy link en_core_web_sm en

from selenium import webdriver
from bs4 import BeautifulSoup

from urllib.parse import urlparse, parse_qs
from collections import OrderedDict

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from collections import OrderedDict
import collections.abc as collections
from collections.abc import Mapping
#from collections import Mapping


!pip install git+https://github.com/Commonists/pageview-api.git

!pip install pageviewapi

#import pageviewapi


In [4]:
%%capture
# This cell will not display any output

!pip install Wikipedia-API
import wikipediaapi


# **Load File**

In [5]:
import pandas as pd

df = pd.ExcelFile("./optimizationOfHintGeneration/testSet.xlsx").parse("Sheet1")
x = []
x.append(df["Answer"])

dataPerson = []
dataYear = []
dataLocation = []

for index, row in df.iterrows():
  if(row["Category"] == "Person"):
    dataPerson.append([row["Question"], row["Answer"]])
  elif(row["Category"] == "Year"):
    dataYear.append([row["Question"], row["Answer"]])
  elif(row["Category"] == "Location"):
    dataLocation.append([row["Question"], row["Answer"]])

person_df = pd.DataFrame(dataPerson, columns=["Question", "Answer"])
year_df = pd.DataFrame(dataYear, columns=["Question", "Answer"])
location_df = pd.DataFrame(dataLocation, columns=["Question", "Answer"])

# **New prediction method for years**

## Dictionary for thumbcaption part of a year



In [6]:
def get_table_info(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    options = webdriver.FirefoxOptions()
    #options.headless = True
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(5) # Wait for the page to load completely
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])
    return (headers, data)

def get_table_info_requests(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    headers = []
    data = []

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])
    return (headers, data)


def get_links_in_section(wikipedia_url, section_heading):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link on that section of the page.
    """
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('span', {'id': section_heading})
    if section is None:
        return None
    section_content = section.parent.find_next_sibling('ul')
    if section_content is None:
        return None
    links = []
    for item in section_content.find_all('li'):
        link = item.find('a')
        if link is not None and link.has_attr('href') and link['href'].startswith('/wiki/'):
            title = link.get('title', '')
            description = item.text.strip()
            url = f"https://en.wikipedia.org{link['href']}"
            links.append({'title': title, 'description': description, 'url': url})
    print(links)
    return links


def get_links_in_section_with_sublinks(wikipedia_url, section_title):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link  with its sublinks as well.
    """
    response = requests.get(wikipedia_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        section_heading = soup.find('span', {'id': section_title})
        if section_heading is None:
            return None
        section = section_heading.parent
        section_content = section.find_next_sibling('ul')
        if section_content is None:
            return None
        links = []
        for item in section_content.find_all('li'):
            link = item.find('a')
            if link is not None:
                link_title = link.get('title')
                link_url = 'https://en.wikipedia.org' + link.get('href')
                link_description = item.text.replace(link_title, '').strip()
                links.append({'title': link_title, 'url': link_url, 'description': link_description})
                for sublink in item.find_all('a', href=True, recursive=False):
                    sublink_title = sublink.get('title')
                    sublink_url = 'https://en.wikipedia.org' + sublink.get('href')
                    sublink_description = sublink.parent.text.replace(sublink_title, '').replace(link_title, '').strip()
                    links.append({'title': sublink_title, 'url': sublink_url, 'description': sublink_description})
        return links
    else:
        return None

"""
Returns all backlinks located in the thumbcaption section
"""
def get_wikipedia_backlinks_thumbcaption(url):
    # Load the Wikipedia page HTML
    page_html = requests.get(url).text
    soup = BeautifulSoup(page_html, 'html.parser')

    # Find the image caption on the page
    caption = soup.find('div', class_='thumbcaption')

    if caption is not None:
      # Extract the caption text and any backlinks
      backlink_sentences = {}
      sentences = caption.text.strip().split('.')
      for sentence in sentences:
        links = caption.find_all('a', href=True, string=re.compile(sentence))
        if len(links) > 0:
            backlinks = []
            for link in links:
                backlink_url = 'https://en.wikipedia.org' + link['href']
                backlink_title = link.get('title', '')
                backlinks.append((backlink_url, backlink_title))
            backlink_sentences[sentence] = backlinks
    return backlink_sentences

"""
This function correctly prunes the links to only include the string after the last / character
"""
def prune_links(links):
    pruned_links = []
    for url, title in links:
        pruned_url = url.split('/')[-1]
        pruned_links.append((pruned_url, title))
    return pruned_links

"""
This function first initializes an empty list combined_list that will hold the combined strings. Then, it uses a for loop to iterate through the input list in increments of 10 tuples at a time.
For each sub-list of up to 10 tuples, it uses the list comprehension and join() method as before to combine the first elements of each tuple into a single string separated by '|'. 
Finally, the function appends each combined string to the combined_list and returns it at the end.
"""
def combine_first_elements(my_list):
    combined_list = []
    num_tuples = len(my_list)
    for i in range(0, num_tuples, 10):
        sub_list = my_list[i:i+10]
        combined_str = '|'.join([tup[0] for tup in sub_list])
        combined_list.append(combined_str)
    return combined_list

"""
This function first initializes an empty list url_list that will hold the modified URLs. Then, it uses a for loop to iterate through each combined string in the input list. 
For each combined string, it concatenates the base URL with a forward slash and the combined string, and appends the resulting URL to the url_list. Finally, the function returns the url_list at the end.
"""
def add_combined_strings_to_url(base_url, combined_strings):
    url_list = []
    for string in combined_strings:
        url_list.append(base_url + '/' + string)
    return url_list


"""
This will output a list of all the sentences in the thumbcaption, split by ';'.
"""
def get_thumbcaption_sentences(url):
    # Get the HTML content of the page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find the thumbcaption element
    thumbcaption = soup.find('div', class_='thumbcaption')

    # Get all the sentences in the thumbcaption
    sentences = thumbcaption.text.split('; ')

    return sentences

"""
This function converts a list into a dict in the way that is needed.
"""
def list_to_dict(lst):
    result = {}
    for sublist in lst:
        if len(sublist) >= 4:
            key = sublist[1]
            value = int(sublist[3].replace(',', ''))
            result[key] = value
    return result

"""
This function takes a list of links, opens each link to get the data, discards the header and converts the data into a dictionary using the list_to_dict() function, 
and then updates a combined dictionary with the resulting dictionary from each link. Finally, it returns the combined dictionary.
"""
def combine_dicts_from_links(link_list):
    combined_dict = {}
    for link in link_list:
        header, data = get_table_info(link)
        #print(data)
        link_dict = list_to_dict(data)
        combined_dict.update(link_dict)
    return combined_dict

"""
This sorts the items in the dictionary based on the integer value of the second element in each key-value tuple (i.e. item[1]), in descending order (reverse=True).
"""
def sort_dict_desc(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: int(item[1]), reverse=True)}

"""
This function takes in the ord dictionary and the sentences list as arguments.
It initializes an empty list called result that we will append the found sentences to. It then iterates over each key in the ord dictionary and for each key, it iterates over each sentence in the sentences list. 
If the key is found in the sentence, the sentence is appended to the result list
"""
def find_sentences(ord, sentences):
    result = []
    for key in ord:
        for sentence in sentences:
            if key in sentence:
                result.append(sentence)
    return result

"""
This function takes a list of sentences and a keyword, removes the keyword from each sentence in the list, and returns the updated list.
"""
def remove_keyword(sentences, keyword):
    updated_sentences = []
    for sentence in sentences:
        updated_sentence = sentence.replace(keyword, "")
        updated_sentences.append(updated_sentence)
    return updated_sentences

"""
This function takes a list of sentences and a list of keyword, removes the keyword from each sentence in the list, and returns the updated list. MAYBE NOT WORKING
"""
def remove_keywords(sentences, keywords):
    """
    Removes one or more keywords from each sentence in the list of sentences.
    """
    result = sentences
    for entry in keywords:
        result = remove_keyword(result, entry)
    
    return result
"""
This function prepends a given string to each sentence in a list.
"""
def prepend_string(sentences, prepend_str):
    new_sentences = []
    for sentence in sentences:
        new_sentence = f"{prepend_str}{sentence}"
        new_sentences.append(new_sentence)
    return new_sentences

"""
This function creates a new list new_sentences and loops over each sentence in the input sentences list. 
It then uses a list comprehension and the difflib.SequenceMatcher class to compare the sentence to each sentence already in new_sentences. 
If the ratio of similarity between the two sentences is greater than 0.8 (adjust this threshold as needed), it considers the sentence to be similar and skips it. Otherwise, it adds the sentence to new_sentences. 
Finally, it returns the new list of unique sentences.
"""


def remove_similar(sentences):
    new_sentences = []
    for sentence in sentences:
        if not any(difflib.SequenceMatcher(None, sentence, s).ratio() > 0.8 for s in new_sentences):
            new_sentences.append(sentence)
    return new_sentences


In [7]:
#IMPORTANT
def thumbcaption_hints_per_year(years_list):
  thumbcaption_hints = {}
  wiki_base_link= 'https://en.wikipedia.org/wiki/'
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=all-time&pages='
  
  for y in years_list:
    years_key = str(y)
    test_link = wiki_base_link + years_key
    sentences_of_thumbcaption = get_thumbcaption_sentences(test_link) #just gets the sentences from the thumbcaption section of a wikipedi years page 
    thumbcaption = get_wikipedia_backlinks_thumbcaption(test_link) #get all backlinks of the thumbcapture of the year (those are the most known events)
    thumbcaption_key = next(iter(thumbcaption))
    thumbcaption_val = thumbcaption[thumbcaption_key]
    pruned = prune_links(thumbcaption_val) #prune those backlinks such that only the important part remains
    com = combine_first_elements(pruned) #combine up to 10 of these links to create a request to pageview
    url_list = add_combined_strings_to_url(pageviews_range_url, com) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page
    print("URL")
    print(url_list)
    data=combine_dicts_from_links(url_list) #now we called the links and retreieved the pageviews; saved them as a dictionary
    ord = sort_dict_desc(data) #now the list is ordered in ascending order
    tmp = find_sentences(ord,sentences_of_thumbcaption) #search the corresponding sentence to the keyword (USA and school shooting for example)
    #remove the years number from the hints OBVIOUSNESS
    keywords_list = [years_key, 'clockwise ', 'Clockwise ', 'From top left', 'from top-left', 'from top left', 'From top-left', 'from top-left: ', ':', 'from left, clockwise'] #list of keywords that should be removed from the sentences
    t4=remove_keywords(tmp, keywords_list)
    prepend_str = 'In the same year, '
    hints = prepend_string(t4, prepend_str)

    final_hints = remove_similar(hints) #before adjusting the sentences
    thumbcaption_hints[y] = final_hints

  return thumbcaption_hints

def get_year_thumbcaption_hints():
  file_years_list = []
  for index, row in year_df.iterrows():
    file_years_list.append(row["Answer"])

  pop_thumb_hints = thumbcaption_hints_per_year(file_years_list)
  return pop_thumb_hints

#get_year_thumbcaption_hints()

## Dictionary for popular sports events of a year

In [8]:
#Some global varaibles for faster execution
pop_year_hints = {}
pop_thumb_hints = {}

def get_all_tables(url):
    """
    Given a URL, this function opens the url and retrieves all tables on the page.
    """
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(1) # Wait for the page to load completely
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    tables = soup.find_all('table')
    all_tables = []
    for table in tables:
        rows = table.find_all('tr')
        headers = [header.text.strip() for header in rows[0].find_all('th')]
        data = []
        for row in rows[1:]:
            data.append([cell.text.strip() for cell in row.find_all('td')])
        all_tables.append({'headers': headers, 'data': data})

    return all_tables

def get_first_elements(data_dict):
    """
    Extracts the first element from each sublist in the 'data' list of lists in a dictionary and returns them in a separate list.
    
    Args:
    - data_dict (dict): A dictionary containing a 'data' key with a list of lists as its value.
    
    Returns:
    - A list containing the first element of each sublist in the 'data' list of lists.
    """
    data_list = data_dict['data']
    result = []
    for sublist in data_list:
        if len(sublist) > 0:
            if len(sublist[0]) > 0:
              result.append(sublist[0])
    return result

#prune the list of the hole wiki page down to the important table
def prune_dict_list(dict_list, keyw):
    for d in dict_list:
        if 'headers' in d and d['headers'] == [keyw]:
            return d
    return None  # if no dict with the desired key-value pair is found

#replace the \n and create a list of lists 
def create_list_from_list_of_lists_key(lst, keyw):
    result = []
    for sublst in lst:
        if sublst:
          new_test = [item.replace(keyw, '') for item in sublst[0].split(keyw)]
          result.append(new_test)
    return result

#replace the : and create a dict; {'year': 'CL-winner'}
def create_dict_from_list_of_lists(lst):
    result_dict = {}
    for sublist in lst:
        for item in sublist:
            if ":" in item:
                key, value = item.split(":")
                key = key.strip().replace("–", "-")
                value = value.strip()
                result_dict[key] = value
            else:
                continue
    return result_dict

#split on the : and creates a dict
def create_dict_from_list(lst):
    result = {}
    for item in lst:
        parts = item.split(":")
       # parts = parts.strip().replace(":", "")
        result[parts[0]] = parts[1]
    return result

#splits on [ and creates a dict 1950: 'Farina'
def get_year_with_driver(race_results):
    results_dict = {}
    for result in race_results:
        if result:
            results_dict[result[0].split("[")[0]] = result[1]
    return results_dict

#get rid of the links ('Alberto Ascari[20]' => 'Alberto Ascari')
def clean_driver_names(results_dict):
    for year, driver in results_dict.items():
        results_dict[year] = driver.split('[')[0].strip()
    return results_dict

#deletes the : from the key
def clean_dict_keys(dict_to_clean):
    cleaned_dict = {}
    for key, value in dict_to_clean.items():
        cleaned_dict[key.rstrip(':')] = value
    return cleaned_dict

#function to split on \n but that lets the city names stay together
def create_city_dict(city_list):
    city_dict = {}
    cities = city_list[0].split('\n')
    for city in cities:
        if city:
            year, *city_name = city.strip().split()
            city_dict[year] = ' '.join(city_name)
    return city_dict


#get the dict of all the champions league winners 
def champions_league_winners_list():
    champions_league_url = 'https://en.wikipedia.org/wiki/List_of_European_Cup_and_UEFA_Champions_League_finals#List_of_finals'
    all = get_all_tables(champions_league_url) #gets all tables of wiki page
    #first prune of that huge dict
    keyw= 'showvteEuropean Cup and UEFA Champions League winners'
    pruned_dict = prune_dict_list(all,keyw)
    #second prune
    inter = pruned_dict.get('data') 
    cl_list = inter[2:5] + inter[7:11]
    tmp1 = create_list_from_list_of_lists_key(cl_list, '\n')
    tmp2 = create_dict_from_list_of_lists(tmp1)

    return tmp2

#get the dict of all the euros winners 
def uefa_euros_winners_list():
    euros_url = 'https://en.wikipedia.org/wiki/List_of_UEFA_European_Championship_finals#List_of_finals'
    all = get_all_tables(euros_url) #gets all tables of wiki page
    #first prune of that huge dict
    keyw= 'showvteUEFA European Championship winners'
    pruned_dict = prune_dict_list(all,keyw)
    #second prune
    inter = pruned_dict.get('data') 
    tmp1 = create_list_from_list_of_lists_key(inter, '\n')
    tmp2 = create_dict_from_list_of_lists(tmp1)

    return tmp2

#get the dict of all the wold cup winners 
def uefa_worlds_winners_list():
    worlds_url = 'https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals#List_of_final_matches'
    all = get_all_tables(worlds_url) #gets all tables of wiki page
    tmp3=get_first_elements(all[3])
    #first prune of that huge dict
    keyw= 'showvteFIFA World Cup'
    pruned_dict = prune_dict_list(all,keyw)
    #second prune
    inter = pruned_dict.get('data') 
    inter = inter[2]
    years = [s for s in inter[0].split('\n')]
    my_dict = dict(zip(years, tmp3))

    return my_dict

#get the dict of all the F1 drivers world champions
def f1_winners_list():
    euros_url = 'https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions#By_season'
    all = get_all_tables(euros_url) #gets all tables of wiki page
    lst = all[2].get('data')
    tmp2 = get_year_with_driver(lst)
    tmp2 = clean_driver_names(tmp2)

    return tmp2

#get the dict of all summer olympics host cities
def summer_olympics_hosts_list():
    summerO_url = 'https://en.wikipedia.org/wiki/Summer_Olympic_Games#List_of_Summer_Olympic_Games'
    all = get_all_tables(summerO_url) #gets all tables of wiki page
    lst = all[10].get('data')
    tmp1 = create_list_from_list_of_lists_key(lst, '\n')
    tmp1 = tmp1[0]
    tmp1 = create_dict_from_list(tmp1)
    tmp1 = clean_dict_keys(tmp1)
    tmp1 = clean_driver_names(tmp1)

    return tmp1

#get the dict of all winter olympics host cities
def winter_olympics_hosts_list():
    winterO_url = 'https://en.wikipedia.org/wiki/Winter_Olympic_Games#List_of_Winter_Olympic_Games'
    all = get_all_tables(winterO_url) #gets all tables of wiki page
    lst = all[8].get('data')
    tmp1 = create_list_from_list_of_lists_key(lst, '\n')
    tmp1 = tmp1[0]
    tmp1 = create_dict_from_list(tmp1)
    tmp1 = clean_dict_keys(tmp1)
    tmp1 = clean_driver_names(tmp1)

    return tmp1

In [9]:
#The basic sentences out of which we create the hint-sentences
basic_sentences =  ['In the same year, ', 'In the previous year, ', 'In the following year, ']
sport_sentences = [' has won the UEFA Champions League.', ' has won the UEFA Euro Football Championship.', ' has won the FIFA World Cup.', ' has won the F1 Drivers World Championship.', ]
olympic_sentences = ['In the same year, the Summer Olympics were held in ', 'In the previous year, the Summer Olympics were held in ', 'In the following year, the Summer Olympics were held in ', 'In the same year, the Winter Olympics were held in ', 'In the previous year, the Winter Olympics were held in ', 'In the following year, the Winter Olympics were held in ']

#write all of the winners of the different sports categories into lists
cl_all = champions_league_winners_list() #For the Champions league, 
euro_all = uefa_euros_winners_list() #For Football-Euros
worlds_all = uefa_worlds_winners_list() #For Football-Worlds
f1_all = f1_winners_list() #For Fromula1
summer_olympics_all = summer_olympics_hosts_list() #For OlympicSummerGames 
winter_olympics_all = winter_olympics_hosts_list() #For OlympicWinterGames 

'''
takes a list of years and then creates a dict of dicts, where (if available) the most popular sports events of that year are saved as hints.
returns a dict with the corresponding sports events from the years in years_list
'''
def popular_sports_per_year(years_list):
 
  pop_sport_hints_year = {}
  for index in years_list:
    year = index
    year_s = str(year)

    year_dict = {
        'cl': '', 'p_cl': '', 'f_cl': '',
        'euros': '', 'p_euros': '', 'f_euros': '',
        'worlds': '', 'p_worlds': '', 'f_worlds': '',
        'f1': '', 'p_f1': '', 'f_f1': '',
        'summer': '', 'p_summer': '', 'f_summer': '',
        'winter': '', 'p_winter': '', 'f_winter': '', 
        }

  # UEFA Champions League: Create the sentences like (In the same-, the following-, the previous-year)
    for key in cl_all:
      if int(key.split('-')[1]) == year % 100:
        result = cl_all[key]
        break
    if result:
      year_dict['cl'] = basic_sentences[0] + result + sport_sentences[0] 
    for key in cl_all:
      if int(key.split('-')[1]) == (year - 1) % 100:
        result = cl_all[key]
        break
    if result:
      year_dict['p_cl'] = basic_sentences[1] + result + sport_sentences[0] 
    for key in cl_all:
      if int(key.split('-')[1]) == (year + 1) % 100:
        result = cl_all[key]
        break
    if result:
      year_dict['f_cl'] = basic_sentences[2] + result + sport_sentences[0]       

  # UEFA EURO Football Championship: Create the sentences like (In the same-, the following-, the previous-year)
    for d in euro_all:
      if year_s in d:
        year_dict['euros'] = basic_sentences[0] + euro_all[year_s] + sport_sentences[1]       
    for d in euro_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_euros'] = basic_sentences[1] + euro_all[year_int] + sport_sentences[1]
    for d in euro_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_euros'] = basic_sentences[2] + euro_all[year_int] + sport_sentences[1]

  # FIFA WORLD Football Championship: Create the sentences like (In the same-, the following-, the previous-year)
    for d in worlds_all:
      if year_s in d:
        year_dict['worlds'] = basic_sentences[0] + worlds_all[year_s] + sport_sentences[2]
    for d in worlds_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_worlds'] = basic_sentences[1] + worlds_all[year_int] + sport_sentences[2]
    for d in worlds_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_worlds'] = basic_sentences[2] + worlds_all[year_int] + sport_sentences[2]

  # F1 WORLD Drivers Championship: Create the sentences like (In the same-, the following-, the previous-year)
    for d in f1_all:
      if year_s in d:
        year_dict['f1'] = basic_sentences[0] + f1_all[year_s] + sport_sentences[3]
    for d in f1_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_f1'] = basic_sentences[1] + f1_all[year_int] + sport_sentences[3]  
    for d in f1_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_f1'] = basic_sentences[2] + f1_all[year_int] + sport_sentences[3]    

  # Summer Olympic Games: Create the sentences like (In the same-, the following-, the previous-year)
    for d in summer_olympics_all:
      if year_s in d:
        year_dict['summer'] = olympic_sentences[0] + summer_olympics_all[year_s] 
    for d in summer_olympics_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_summer'] = olympic_sentences[1] + summer_olympics_all[year_int] 
    for d in summer_olympics_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_summer'] = olympic_sentences[2] + summer_olympics_all[year_int]  

  # Winter Olympic Games: Create the sentences like (In the same-, the following-, the previous-year)
    for d in winter_olympics_all:
      if year_s in d:
        year_dict['winter'] = olympic_sentences[3] + winter_olympics_all[year_s] 
    for d in winter_olympics_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_winter'] = olympic_sentences[4] + winter_olympics_all[year_int] 
    for d in winter_olympics_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_winter'] = olympic_sentences[5] + winter_olympics_all[year_int]  

    #write the entry in the dict
    pop_sport_hints_year[year] = year_dict
  
  return pop_sport_hints_year

#CALL FUNCTION
#returns a dictionary of the years that occured in the xls file, with its corresponding hints from the most popular sports events of that year.
def get_year_sports_hints():
  file_years_list = []
  for index, row in year_df.iterrows():
    file_years_list.append(row["Answer"])
  pop_sport_hints = popular_sports_per_year(file_years_list)
  
  return pop_sport_hints


In [10]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def preprocess_text(text):
    # Tokenize the text and add special tokens
    tokens = tokenizer.encode(text, add_special_tokens=True)
    # Convert the tokens to a tensor
    token_tensor = torch.tensor(tokens).unsqueeze(0)
    return token_tensor

def get_similarity_score(text1, text2):
    # Preprocess both texts
    tensor1 = preprocess_text(text1)
    tensor2 = preprocess_text(text2)

    # Pass both tensors to the model to get the embeddings
    with torch.no_grad():
        output1 = model(tensor1)
        output2 = model(tensor2)
    
    # Compute the cosine similarity between the two embeddings
    cosine_sim = torch.nn.functional.cosine_similarity(output1.last_hidden_state.mean(dim=1), output2.last_hidden_state.mean(dim=1), dim=1)
    return cosine_sim.item()

## Test it!

In [11]:
# create the hints for the questions where the answer is a year and print them
#if len(pop_year_hints) == 0 and len(pop_thumb_hints) == 0:
pop_year_hints = get_year_sports_hints()
pop_thumb_hints = get_year_thumbcaption_hints()

years_hints = {}

for y in pop_year_hints:
  year_dict = {
      'sports': pop_year_hints[y],
      'thumbcaption': pop_thumb_hints[y]
  }

  years_hints[y] = year_dict

generated_hints_for_years = years_hints
pprint.pprint(generated_hints_for_years, indent=1)

'''
#Test for utility score of new questions; calculate score via BERT for each question,hint pair and write the score together with the question into the sim_scores dictionary.
'''

qa_dict = dict(zip(year_df['Answer'], year_df['Question']))
sim_scores = years_hints

for y, q in qa_dict.items():
  for year, data in years_hints.items():
    if y == year:
      for category, subdata in data.items():
        #sim_scores[year]['question'] = q
        if category != 'thumbcaption':
          for key, value in subdata.items():
            sim_scores[year][category][key] = {}
            similarity_score = get_similarity_score(q,value)
            sim_scores[year][category][key][value] = similarity_score
        else:
          for i in subdata:
            sim_scores[year][category] = {}
            similarity_score = get_similarity_score(q,i)
            sim_scores[year][category][i] = similarity_score
    else:
      continue

for y, q in qa_dict.items():
  for year, data in years_hints.items():
    if y == year:
      sim_scores[year]['question'] = q

pprint.pprint(sim_scores, indent=1)

{}
{}


## Dictionary for thumbcaption part of a year



# **New prediction Methods for people:**

We want to create some sort of metric to explain how popular or well-known a entity is, to give better hints. Therefore we look at how many options there are (how many entries in a certain category) and at how diverse these options are. If a category for example has 100s of entries, then it probably isn't a good base for a hint because there are many to choose from.



## Reusable functions to retrieve, compare and modify the needed data 

In [12]:
"""
Given a list of Wikipedia category names, creates the corresponding Wikipedia category links.
Args: categories (list): A list of Wikipedia category names to create links for.
Returns: category_links (list): A list of Wikipedia category links corresponding to the input categories.
"""
def get_category_links(categories):
  category_links = []
  for category in categories:
    category_link = "https://en.wikipedia.org/wiki/Category:" + category.replace(" ", "_")
    #category_link = "https://wikipedia.org/wiki/Category:" + category.replace(" ", "_")
    category_links.append(category_link)
  return category_links

"""
Given a list of Wikipedia category names, creates the corresponding Wikipedia category links.
Args: categories (list): A list of Wikipedia category names to create links for.
Returns: category_links (list): A list of Wikipedia category links corresponding to the input categories.
"""
def get_category_with_underscores(categories):
  
  category_links = []
  for category in categories:
    category_link = category.replace(" ", "_")
    category_links.append(category_link)
  return category_links

"""
Given a list of Wikipedia category names, retrieves the number of entries in each category and returns a dictionary
with the category names as keys and the entry counts as values.
Args: categories (list): A list of Wikipedia category names to retrieve entry counts for.
Returns: category_entry_counts (dict): A dictionary of Wikipedia category names and their respective entry counts.
"""
def get_category_entry_counts(categories):
  category_links = get_category_links(categories)
  category_entry_counts = {}

  for i, category_link in enumerate(category_links):
    response = requests.get(category_link)
    soup = BeautifulSoup(response.content, "html.parser")
    try:
      entry_count_text = soup.find("div", {"id": "catlinks"}).find_all("a")[1].text.strip()
      entry_count = int(entry_count_text.split()[-2])
      category_name = categories[i]
      if category_name:
        category_entry_counts[category_name] = entry_count
    except:
      pass

  return category_entry_counts

# SPLIT OF 1. CELL


In [13]:
# rewrite with wikiapi
"""
Retrieves the categories of a Wikipedia page using the wikipediaapi package.
Args: title (str): The title of the Wikipedia page.
Returns: categories (list): A list of categories associated with the page.
"""
def get_wikipedia_categories(title):
  wikipedia = wikipediaapi.Wikipedia("en")
  page = wikipedia.page(title)
  #print(page)
  if not page.exists():
    return []
  categories = [c for c in page.categories]
  return [cat.split(":")[1] for cat in categories]

# SPLIT OF 2. CELL

def get_category_subcategories(link):
  #print(link)
  # Create a Wikipedia API object
  wiki_api = wikipediaapi.Wikipedia('en')
  # Extract the category name from the link
  category_name = link.split('/')[-1]
  # Retrieve the category page
  category_page = wiki_api.page(f"Category:{category_name}")
  # Find the subcategories section of the page
  subcategories_section = category_page.categorymembers
  # Extract the subcategories from the section
  subcategories = []
  i=0
  for subcategory in subcategories_section.values():
    if i < 100: #threshold for how many entries per category
      i +=1
      if subcategory.ns == wikipediaapi.Namespace.CATEGORY:
        subcategories.append(subcategory.fullurl)
  return subcategories


def get_category_pages(category_title, limit=100):
  params = {
    "action": "query",
    "format": "json",
    "list": "categorymembers",
    "cmtitle": category_title,
    "cmlimit": str(limit) # limit to 100 entries
  }
  #print("OK")
  pages = []
  while True:
    response = requests.get("https://en.wikipedia.org/w/api.php", params=params).json()
    pages += [p["title"] for p in response["query"]["categorymembers"]]
    if "continue" in response:
      params.update(response["continue"])
    else:
      break
    if len(pages) >= limit: # break the loop if the number of entries exceeds 100
      break
  return pages[:limit] # return only the first 100 entries

# SPLIT OF 3. CELL


In [14]:

"""
Given a list of Wikipedia category names, retrieves the number of entries in each category and returns a dictionary
with the category names as keys and the entry counts as values.
Args: searched_location: The desired location for wich a hint should be created. (The answer to the question)
Returns: dicto (dict): A dictionary of all categories with the subcategories and how many entries there are in each.
"""
def get_cat_with_all_subcats(searched_location):
  #retrieves the list of categories from the location-wikipedia page
  categories = get_wikipedia_categories(searched_location)
  categories_with_underscore = get_category_with_underscores(categories)
  categories_links = get_category_links(categories)

  categories_with_links_dict = {}
  for i in range(len(categories_with_underscore)):
    key = categories_with_underscore[i]
    link = categories_links[i]
    categories_with_links_dict[key] = link

  cat_with_subcats_dict = {}
  for category in categories_links:
    sub_cats = get_category_subcategories(category)
    #print(category)
    ct = get_category_title(category)
    #print(ct)
    pages_list = get_category_pages(ct)
    #print(pages_list)
    filtered_list = [str(entry) for entry in pages_list if not entry.startswith("Category:")]
    new_list = [len(filtered_list), filtered_list]
    if sub_cats is None:
      continue
    else:
      cat_with_subcats_dict[category] = [[len(sub_cats), sub_cats], new_list]

  return cat_with_subcats_dict

#input a url of a category, this returns the tilte
def get_category_title(category_url):
  parts = category_url.split('/')
  title = [part for part in parts if part.startswith('Category:')]
  if title:
    return title[0]
  else:
    return None

#input searched location and returns a dict with number of pages and number of subcategories
def get_categories_ranking(searched_location):
  categories = get_wikipedia_categories(searched_location)
  categories_links = []
  cat_without_articles = []

  bad_list = ['Articles with', 'CS1', 'Wikipedia', 'Webarchive', 'Short', 'Biography', 'Commons', 'Pages', 'Use', 'All', 'Articles', 'Coordinates', 'Engvar', 'Lang', 'Official']

  for c in categories:
    if not any(c.startswith(word) for word in bad_list):
      cat_without_articles.append(c)  

  categories_links = get_category_links(cat_without_articles)
  cat_with_amount = {}

  for category in categories_links:
    sub_cats = get_category_subcategories(category)
    ct = get_category_title(category)
    pages_list = get_category_pages(ct)

    if sub_cats is None:
      continue
    else:
      cat_with_amount[category] =  len(pages_list), len(sub_cats)
  #x[0] sort after subcats and x[1] sorts after pages
  sorted_dict = dict(sorted(cat_with_amount.items(), key=lambda x: x[1], reverse=True))
  return sorted_dict

#NEW
#extract the category part from the wiki links
def extract_last_parts(links):
  last_parts = []
  for link in links:
    last_part = link.split('/')[-1]
    last_parts.append(last_part)
  return last_parts

#function to concatenate the category links to insert into a pageviews url
def concatenate_elements(elements, n=10):
  result = []
  for i in range(0, len(elements), n):
    group = elements[i:i+n]
    result.append('|'.join(group))
  return result

"""
This function first initializes an empty list url_list that will hold the modified URLs. Then, it uses a for loop to iterate through each combined string in the input list. 
For each combined string, it concatenates the base URL with a forward slash and the combined string, and appends the resulting URL to the url_list. Finally, the function returns the url_list at the end.
"""
def combine_pv_urls(base_url, combined_strings):
    url_list = []
    for strin in combined_strings:
      url_list.append(base_url  + strin)
    return url_list


"""
This function takes a list of links, opens each link to get the data, discards the header and converts the data into a dictionary using the list_to_dict() function, 
and then updates a combined dictionary with the resulting dictionary from each link. Finally, it returns the combined dictionary.
"""
def combine_dicts_from_links(link_list):
  combined_dict = {}
  for link in link_list:
    header, data = get_table_info(link)
    #print(data)
    link_dict = list_to_dict(data)
    combined_dict.update(link_dict)
  return combined_dict

# SPLIT OF 4. CELL



In [15]:
def add_values_to_links(links_dict, values_dict):
  new_dict = {}
  for link, value in links_dict.items():
    key = link.split(':')[-1]
    if key in values_dict:
      value += (values_dict[key],)
    new_dict[link] = value
  return new_dict

def combine_catnumbers_pvs(ord_dict, norm_dict):
  for key in ord_dict:
    if key in norm_dict:
      ord_dict[key] = ord_dict[key] + (norm_dict[key],)
      #print(ord_dict)
  return ord_dict

def add_values_to_linkss(links_dict, values_dict):
  new_dict = {}
  for loc, ord_list in links_dict.items():
    for loc2, ord_list_pv in values_dict.items():
      if loc == loc2:
        new_dict[loc] = combine_catnumbers_pvs(ord_list, ord_list_pv)
        #new_dict[loc] = combine_catnumbers_pvs(ord_list_pv, ord_list)
  return new_dict

#combines the pageviews of the categories of the location together with the sub-categories and pages of those subcategories
def combine_pv_cats(cat_dict, pv_dict):
  tmp = cat_dict

  for key, value in cat_dict.items():
    for key2, value2 in pv_dict.items():
      category_name = key.split('/')[-1]
      new_string = key2.replace(' ', '_')

      if category_name == new_string:
        #print(category_name)
        org_tup = cat_dict[key]
        #new_tup = org_tup + (value2, 0)
        new_tup = org_tup + (value2,)
        tmp[key] = new_tup

        if len(new_tup) != 3:
          print("PROBLEM\n")

  return tmp

#combines the pageviews of the categories of the location together with the sub-categories and pages of those subcategories
def get_dict_for_every_location(cat_ranking, cat_with_pv):
  ret=cat_ranking

  for location, value in cat_ranking.items():
    for location1, value1 in cat_with_pv.items():
      if location == location1:
        tmp = combine_pv_cats(value, value1)
        ret[location] = tmp
  return ret

In [66]:
import wikipediaapi
#import pageviewapi

def get_categories(subject_dict):
  categories_for_subject_dict= {}
  rankings_for_categories_dict = {}
  #creates a dict with all the dicts of each location with its categories and sub-categories
  for subject, question in subject_dict.items():
    dicto = get_cat_with_all_subcats(subject)
    categories_for_subject_dict[subject] = dicto

    print("categories_for_subject_dict")
    #pprint.pprint(categories_for_subject_dict)

    ranking = get_categories_ranking(subject)

    print("get_categories_ranking")
    #pprint.pprint(ranking)

    ordict = OrderedDict(ranking)
    rankings_for_categories_dict[subject] = ordict

    print("rankings_for_categories_dict")
    #pprint.pprint(rankings_for_categories_dict)
   
  return rankings_for_categories_dict

#given a dictionary with all the categories, the function returns the categories in a OrderedDict with the corresponding pageviews for each category
def get_pageviews_for_categories(cat_dict):
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=last-year&pages='
  
  all_cats_with_pvs = {}
  if len(cat_dict) == 0:
    print("empty")

  for subject in cat_dict:
    ord_dict = OrderedDict()
    ordered_dict_sub =  cat_dict[subject]
    links_list = [link for link in ordered_dict_sub.keys()]
    #print(links_list)
    pruned_link_parts_list = extract_last_parts(links_list)
    concat_str_for_links = concatenate_elements(pruned_link_parts_list) #combine up to 10 of these links to create a request to pageview
    pageviews_url_list = combine_pv_urls(pageviews_range_url, concat_str_for_links) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page (REUSED FROM YEARS PART)
    #print(pageviews_url_list)
    categories_with_pageviews =combine_dicts_from_links(pageviews_url_list) #now we called the links and retreieved the pageviews; saved them as a dictionary
    #print(categories_with_pageviews)
    ordered_categories_with_pageviews = sort_dict_desc(categories_with_pageviews) #now the list is ordered in ascending order
    #all_cats_with_pvs[loc] = OrderedDict(ordered_categories_with_pageviews)
    all_cats_with_pvs[subject] = OrderedDict(categories_with_pageviews)
  return all_cats_with_pvs

def sorting_dict(sor_dict):
  ret = {}
  l2 = sor_dict
  for loc, value in l2.items():
    if len(sor_dict) == 0:
      print("empty")
      continue
    try:
      sorted_dict = OrderedDict(sorted(value.items(), key=lambda x: x[1][2], reverse=True))
    except Exception as e:
      print(f"Sorting failed for person {loc}: {e}")
      pprint.pprint(value)
      sorted_dict = value
    ret[loc] = sorted_dict
    #sorted_dict = dict(sorted(value.items(), key=lambda x: x[2], reverse=True))
  return ret


from collections import OrderedDict
# function that takes a dictionary with an ordered dictionary as the value and prunes the ordered dictionary to keep only the first n entries:
def prune_ordered_dict(dictionary, n):
  pruned_dict = OrderedDict()
  for key, value in dictionary.items():
    pruned_dict[key] = OrderedDict(list(value.items())[:n])
  return pruned_dict


# function that takes a dictionary with an ordered dictionary as the value and prunes the ordered dictionary to keep only the first n entries and deltes certain categories:
def prune_and_ordered_dict(dictionary, n):
  pruned_dict = OrderedDict()
  inter1_dict= OrderedDict()
  for key, value in dictionary.items():
    inter3_dict= OrderedDict()
    for link, tuplee in value.items():
      link_str = str(link)
      if 'Living_people' not in link_str and '_births' not in link_str and '_deaths' not in link_str and 'Good_articles' not in link_str and 'Members' not in link_str and '20th' not in link_str and '21st' not in link_str:
        inter3_dict[link] = tuplee
    pruned_dict[key] = inter3_dict

  for key, value in pruned_dict.items():
    inter1_dict[key] = OrderedDict(list(value.items())[:n])
  return inter1_dict


#find the 20 most appearing categories 
def find_most_common_links(data_dict):
  link_count = {}
  for key, value in data_dict.items():
    for link in value:
      if link not in link_count:
        link_count[link] = {"count": 1, "keys": [key]}
      else:
        link_count[link]["count"] += 1
        link_count[link]["keys"].append(key)

  # Sort the links by their count in descending order
  sorted_links = sorted(link_count.items(), key=lambda x: x[1]["count"], reverse=True)

  # Return a list of tuples with the link, count, and keys
  return [(link, data["count"], data["keys"]) for link, data in sorted_links[:20]]

In [17]:
template_sentence_location = 'The location you are looking for, is a member of the category x'
template_sentence_location2 = 'The location you are looking for, belongs to the category '

template_sentence_person = 'The person you are looking for, is a member of the category x'
template_sentence_person2 = 'The person you are looking for, belongs to the category '

import itertools
import requests

def get_categories_with_pageviews_person(person_questions_dict):
  cat_ranking_person = get_categories(person_questions_dict)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  #for person: sorting the dict after pages per category
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  #pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)
  return new_ordered_dict_person

#retrives the category name from the link
def get_category_name(link):
  category_prefix = "Category:"
  if link.startswith("https://wikipedia.org/wiki/"):
    return link[len("https://wikipedia.org/wiki/" + category_prefix):].replace("_", " ")
  else:
    if link.startswith("https://en.wikipedia.org/wiki/"):
      return link[len("https://en.wikipedia.org/wiki/" + category_prefix):].replace("_", " ")
    else:
      return None
      
#just get the first 3 categories without any special formula
def get_first_three_categs_location(copy_categories_with_subs_and_pageviews_location):
  first_three_categories_per_location = {}
  for key, value in copy_categories_with_subs_and_pageviews_location.items():
    first_three = dict(itertools.islice(value.items(), 3))
    first_three_categories_per_location[key] = first_three

  hint_sentence_location = {}
  for key, value in first_three_categories_per_location.items():
    inter_hints = []
    for link, tup in value.items():
      cat_name=get_category_name(link)
      if cat_name is not None:
        ok = template_sentence_location2 + cat_name
      else: 
        ok = template_sentence_location2
      inter_hints.append(ok)
    hint_sentence_location[key] = inter_hints
  return hint_sentence_location

#just get the first 3 categories without any special formula
def get_first_three_categs_person(copy_categories_with_subs_and_pageviews_location):
  first_three_categories_per_location = {}
  for key, value in copy_categories_with_subs_and_pageviews_location.items():
    first_three = dict(itertools.islice(value.items(), 3))
    first_three_categories_per_location[key] = first_three

  hint_sentence_location = {}
  for key, value in first_three_categories_per_location.items():
    inter_hints = []
    for link, tup in value.items():
      cat_name=get_category_name(link)
      if cat_name is not None:
        ok = template_sentence_person2 + cat_name
      else: 
        ok = template_sentence_person2
      inter_hints.append(ok)
    hint_sentence_location[key] = inter_hints
  return hint_sentence_location

#get predicates of a wiki page via wikidataAPI
def get_wikidata_predicates(page_title):
    # First, get the Wikidata item ID of the page
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles={page_title}&format=json"
    response = requests.get(url).json()
    pages = response["query"]["pages"]
    if "-1" in pages:
        return None
    page_id = next(iter(pages))
    wikidata_id = pages[page_id]["pageprops"]["wikibase_item"]

    # Then, get the predicates and their values of the Wikidata item
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={wikidata_id}&format=json&props=claims"
    response = requests.get(url).json()
    entity = response["entities"][wikidata_id]
    predicates = {}
    for claim_id, claim in entity["claims"].items():
        predicate_id = claim["mainsnak"]["property"]
        predicate_value = claim["mainsnak"]["datavalue"]["value"]
        predicates[predicate_id] = predicate_value
    return predicates

#TEST
people_list=[]
for l in dataPerson:
  people_list.append(l[1])

# takes a list of all the people and the list of category occurences where all the categories that were assigned to these wiki-pages 
# are listed and ranked after how often they occur; 
# output: a list where the person is the key and the entries show wich categories the entity shares with wich other entitities and how many they are
def get_people_dict(people_list, occurences_list):
  people_dict = {}
  for person in people_list:
    person_categories = []
    for occurence in occurences_list:
      if person in occurence[2]:
        person_categories.append((occurence[0], occurence[1], occurence[2]))
    people_dict[person] = person_categories
  return people_dict

#returns a dict where for each person in people_list we calculate how often they occur in the same category as the keys in person_dict 
# (where already for each person we looked at each of the categories he appears and counted how many other people from people_list appear in these )
def count_people_occurrences(person_dict, people_list):
  # Initialize an empty dictionary to hold the counts  
  count_dict = {}
  # Loop through all pairs of drivers and count the number of occurrences
  for key, value in person_dict.items():
    counter = {}
    for person in people_list:
      counter[person] = 0
      for cat in value:
        if person in cat[2]:
          counter[person] += 1  
    count_dict[key] = counter
  return count_dict

#sorting
def sort_dict_by_value_desc(d):
  # Sort the inner dictionary by value in descending order
  sorted_dict = []
  sorted_dict = OrderedDict(sorted(d.items(), key=lambda x: x[1], reverse=True))
  return sorted_dict

def get_categories_union(overlap_dict, people_list):
  # Initialize an empty dictionary to hold the counts  
  count_dict = {}
  for key, value in overlap_dict.items():
    counter = {}
    for item, number in value.items():
      #print(number)
      counter[item] = number_categories_per_person[key] + number_categories_per_person[item] - number
    count_dict[key] = counter
  return count_dict

def get_avg_pairwise_sim(cat_div_union, cat_div_overlap, number_categories_per_person, people_list):
  count_dict = {}
  for key, value in cat_div_union.items():
    inter = {}
    for key1, value1 in cat_div_overlap.items():
      if key == key1:
        for name, number in value.items():
          for name1, number1 in value1.items():
            if name == name1:
              if not number  or not number1:
                inter[name] = 0
              else:
                inter[name] = (number1 / number)
    count_dict[key] = inter
  return count_dict

def get_cat_diversity(shared_categories, copy_new_ordered_dict_person):
  count_dict = {}
  for key, value in copy_new_ordered_dict_person.items():
    for item in value.items():
      pv = 0
      #print(item)
      link = item[0]
      trip = item[1]
      if len(trip) == 3:
        pv = trip[2]
      if link not in count_dict:
        count_dict[link] = pv
  return count_dict


In [18]:


#---- SPLIT ----

#shared_categories; number_categories_per_person; unsorted_category_diversity; cat_div_overlap; unsorted_cat_div_union; cat_div_union; unordered_avg_pairwise_sim; avg_pairwise_sim; unordered_cat_popularity;cat_popularity

def get_categories_with_ranking():
  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
  cat_with_pv_person = {}

  cat_ranking_person = get_categories(person_questions_dict)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)

  return new_ordered_dict_person

#all_people_cat_ranked = get_categories_with_ranking() #copy_new_ordered_dict_person

import wikipediaapi
import requests

#given a name, retrieve the infomration in the short-description part of the wiki page
def get_page_short_description(page_title):
  # Prepare the API request URL
  url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title.replace(' ', '_')}"
  # Send the API request
  response = requests.get(url)
  data = response.json()
  # Extract the short description from the API response
  short_description = data.get('description', '')
  return short_description

def find_most_similar_category(persons, specific_category, num_similar_categories=3):
  most_similar_categories = {}

  for person, categories in persons.items():
    category_texts = [category_link.split('/')[-1].replace('_', ' ') for category_link in categories.keys()]
    category_texts.append(specific_category[person])  # Add the specific category for comparison

    # Vectorize the category texts
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(category_texts)
    #print(category_texts)
    # Calculate cosine similarity between the specific category and other categories
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
      # Calculate cosine similarity between the specific category and other categories
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
    # Find the most similar categories textually
    similar_indices = similarities.argsort()[-num_similar_categories:][::-1]
    similar_categories = [category_texts[index] for index in similar_indices]
    most_similar_categories[person] = similar_categories
  return most_similar_categories


def get_work_category(similar_categories):
  person_with_work_categories = {}
  person_with_work_category = {}
  strings_to_check = ["births", "deaths"]
  for person, categories in similar_categories.items():
    for strs in strings_to_check:
      for cats in categories:
        if strs in cats:
          categories.remove(cats)
        else:
          continue
      person_with_work_categories[person] = categories
  for person, categories in person_with_work_categories.items():
    person_with_work_category[person] = categories[0]
  return person_with_work_category


def get_container_categories(work_cats):
  for person, categories in work_cats.items():
    wiki_category = categories
    base_url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'titles': wiki_category,
        'prop': 'categories',
        'format': 'json',
        'cllimit': 'max'
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    page_id = next(iter(data['query']['pages'].keys()))
    categories = data['query']['pages'][page_id].get('categories', [])
    container_categories = []
    for category in categories:
        if category['title'].startswith('Category:'):
            container_categories.append(category['title'][9:])

    strings_to_check = ["CatAutoTOC ", "Commons", "Template"]
    for strs in strings_to_check:
      for cats in container_categories:
        if strs in cats:
          container_categories.remove(cats)
        else:
          continue
    work_cats[person] = container_categories
  return work_cats

def replace_spaces_with_underscore(dictionary):
  updated_dict = {}
  for key, value in dictionary.items():
    updated_dict[key] = value.replace(" ", "_")
  return updated_dict

def format_category_dict(category_dict):
  formatted_dict = {}
  for key, value in category_dict.items():
    if isinstance(value, list):
      formatted_value = [f"Category:{v.replace(' ', '_')}" for v in value]
    else:
      formatted_value = f"Category:{value.replace(' ', '_')}"
    formatted_dict[key] = formatted_value
  return formatted_dict


In [19]:
#---- SPLIT ----

#person_with_job; similar_categories; work_category; container_category_work; test2

"""
Functions to retrieve the occupation of a person

The wikipedia-api library does not provide a direct method to retrieve the infobox of a Wikipedia page. 
However, we can use beautifulsoup4 to parse the HTML content of the Wikipedia page and extract the infobox.
"""

#function that retrieves the infobox using beautifulsoup4
def get_infobox_from_wikipedia(page_title):
  # Format the page title for the Wikipedia URL
  formatted_title = page_title.replace(' ', '_')
  # Construct the Wikipedia page URL
  url = f"https://en.wikipedia.org/wiki/{formatted_title}"
  # Send a GET request to the Wikipedia page
  response = requests.get(url)
  # Check if the page exists
  if response.status_code != 200:
    print(f"Page '{page_title}' does not exist.")
    return None
  # Create a BeautifulSoup object to parse the page content
  soup = BeautifulSoup(response.content, 'html.parser')
  # Find the infobox section of the page
  infobox = soup.find(class_='infobox')
  return infobox

#function that retrieves the infobox from a Wikipedia page and specifically extracts the entries from the "occupation" or "occupations" field:
def get_occupations_from_infobox(page_title):
  # Retrieve the infobox from the Wikipedia page
  infobox = get_infobox_from_wikipedia(page_title)
  # Check if the infobox exists
  if not infobox:
    return []
  # Search for the "occupation" or "occupations" field in the infobox
  occupation_keywords = ['occupation', 'occupations', 'Occupation', 'Occupations']
  occupations = []
  for keyword in occupation_keywords:
    field = infobox.find(string=keyword)
    if field:
      # Retrieve the occupation entries from the field
      tmp_str = field.find_next("td").find_next("div").find_next("ul")
      from_table = extract_list_elements(tmp_str)
      from_field = field.find_next("td").text.strip()
      from_field = from_field.split()
      if len(from_table) > 0:
        for i in from_table:
          occupations.append(i)
      elif len(from_field) > 0:
        for i in from_field:
          occupations.append(i)
      else:
        print("problem")
  return occupations

#extract the elements between the <li> tags
def extract_list_elements(html_string):
  occupations_list = []
  li_entries_list = []
  html_str = str(html_string)
  html_str = html_str.replace("</ul>", "")
  html_str = html_str.replace("<ul>", "")
  html_str = html_str.replace("</li>", "")
  li_entries_list = html_str.split("<li>")
  for i in li_entries_list:
    if len(i) > 0: #check if empty
      if len(i) < 16: #check if entry is a link
        occupations_list.append(i)
  return occupations_list

#searches the occupation for every entry in people list
def get_occupations(people_list):
  occupation_person_dict = {}
  for person in people_list:
    occupation_person_dict[person] = get_occupations_from_infobox(person) 
  return occupation_person_dict

#people_list; occupation_person_dict

In [20]:
#---- SPLIT ----

#retrieves all of the relate links of a wiki page
def get_related_links(wiki_link):
  # Extract the page title from the Wikipedia link
  title = wiki_link.split('/')[-1]
  # Format the API URL to get the page content
  api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=links&pllimit=max&format=json'
  # Send a GET request to the API
  response = requests.get(api_url)
  if response.status_code == 200:
    data = response.json()
    # Extract the page ID from the API response
    page_id = next(iter(data['query']['pages']))
    # Check if the page exists and has links
    if page_id != '-1' and 'links' in data['query']['pages'][page_id]:
      # Retrieve the links from the API response
      links = data['query']['pages'][page_id]['links']
      # Extract the link titles and URLs
      related_links = [{'url': f'https://en.wikipedia.org/wiki/{link["title"]}', 'title': link['title']} for link in links]
      return related_links
  return []

# function to check if the title consists of two words -> discrad a lot of entries for performance reasons
def filter_two_word_titles(links):
  filtered_links = []
  url_concat = ""
  for link in links:
    url = link['url']
    link['url'] = url.replace(' ', '_')
    title = link['title']
    words = title.split()
    if len(words) == 2:
      filtered_links.append(link)

  return filtered_links

#check if a given Wikipedia link corresponds to an entity with the "instance of" (P31) property set to "human" (Q5), indicating it represents a person
def is_person_page(link):
  # Extract the page title from the link
  title = link['title']
  # Query Wikidata API to check if the page corresponds to a person
  wikidata_url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&sites=enwiki&titles={title}"
  response = requests.get(wikidata_url).json()
  # Check if the response contains any entities
  if 'entities' in response:
    entities = response['entities']
    # Check if there is an entity with P31 (instance of) set to Q5 (human)
    for entity_id, entity in entities.items():
      if 'claims' in entity and 'P31' in entity['claims']:
        for claim in entity['claims']['P31']:
          if claim['mainsnak']['datavalue']['value']['id'] == 'Q5':
            return True
  return False

#function that calls the is_perso_page() function for every link in the pre discraded list
def check_if_person(links):
  person_links= []
  for link in links:
    if is_person_page(link):
      person_links.append(link)
  return person_links

#function that selects 5 entries at random from a list of related people:
def select_random_entries(related_people_list, num_entries=5):
  random_entries = random.sample(related_people_list, num_entries)
  return random_entries

# takes a list of all the people and the list of category occurences where all the categories that were assigned to these wiki-pages 
# are listed and ranked after how often they occur; 
# output: a list where the person is the key and the entries show wich categories the entity shares with wich other entitities and how many they are
def get_people_dict(people_list, occurences_list):
  people_dict = {}
  for person in people_list:
    person_categories = []
    for occurence in occurences_list:
      if person in occurence[2]:
        person_categories.append((occurence[0], occurence[1], occurence[2]))
    people_dict[person] = person_categories
  return people_dict

#wiki_link; related_articles; filtered_links; related_people_list; select_random_people; 


def get_most_known_related_people(related_people_list):
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=last-year&pages='
  ord_dict = OrderedDict()
  link_list = []
  for item in related_people_list:
    link_list.append(item['url'])
  pruned_link_parts_list = extract_last_parts(link_list)
  concat_str_for_links = concatenate_elements(pruned_link_parts_list) #combine up to 10 of these links to create a request to pageview
  pageviews_url_list = combine_pv_urls(pageviews_range_url, concat_str_for_links) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page (REUSED FROM YEARS PART)
  categories_with_pageviews =combine_dicts_from_links(pageviews_url_list) #now we called the links and retreieved the pageviews; saved them as a dictionary
  ordered_categories_with_pageviews = sort_dict_desc(categories_with_pageviews) #now the list is ordered in ascending order
  ord_dict = ordered_categories_with_pageviews

  return ord_dict

#tester; 
#related_people_ranked; top_most_popular_people; cat_ranking_related_person; cat_with_pv_person; categories_with_subs_and_pageviews_person
#new_ordered_dict_related_person; copy_new_ordered_dict_person_test; ordered_dict_related_person

In [47]:
#---- SPLIT ----


def get_ordered_categories_of_most_related_people(most_related_peoples_list):
  #tester = get_most_known_related_people()
  ret = {}
  for k,v in most_related_peoples_list.items():
    related_people_ranked = dict(sorted(v.items(), key=lambda x: x[1], reverse=True))
    top_most_popular_people = dict(itertools.islice(related_people_ranked.items(), 5))#take the top 5 most known people from the list
    
    cat_ranking_related_person = get_categories(top_most_popular_people)
    print(cat_ranking_related_person)
    cat_with_pv_person = get_pageviews_for_categories(cat_ranking_related_person)
    print(cat_with_pv_person)
    #categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
    categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_related_person, cat_with_pv_person)
    print(categories_with_subs_and_pageviews_person)

    new_ordered_dict_related_person = sorting_dict(categories_with_subs_and_pageviews_person)
    
    copy_new_ordered_dict_person_test = prune_and_ordered_dict(new_ordered_dict_related_person, 20)
    ordered_dict_related_person = sorting_dict(copy_new_ordered_dict_person_test)
    ret[key] = ordered_dict_related_person
  return ret

#get the links of all answer-entities via link creation of the key
def get_rel_pep():
  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
  wiki_link = 'https://en.wikipedia.org/wiki/'
  ret = {}

  for key,value in person_questions_dict.items():
    modified_text = key.replace(' ', '_')
    link = f"{wiki_link}/{modified_text}" 
    related_articles = get_related_links(link)
    filtered_links = filter_two_word_titles(related_articles)
    related_people_list = check_if_person(filtered_links)
    ret[key] = related_people_list
  return ret

#a; inter; intermediate

#get the related people with the categories
def get_related_with_categories():
  ret = {}
  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))

  cat_ranking_person = get_categories(person_questions_dict)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)

  for key,value in new_ordered_dict_person.items():
    ret[key] = value
    
  return ret

## Print and Test the functions!

In [22]:
#for locations

location_questions_dict = dict(zip(location_df['Answer'], location_df['Question']))
cat_with_pv_location = {}

#for locations
cat_ranking_location = get_categories(location_questions_dict)
#pprint.pprint(cat_ranking_location, indent=1)
cat_with_pv_location = get_pageviews_for_categories(cat_ranking_location)
#pprint.pprint(cat_with_pv_location, indent=1)
#for locations: sorting the dict after pages per category
categories_with_subs_and_pageviews_location = get_dict_for_every_location(cat_ranking_location, cat_with_pv_location)
#pprint.pprint(categories_with_subs_and_pageviews_location,indent=1)
new_ordered_dict_location = sorting_dict(categories_with_subs_and_pageviews_location)
#pprint.pprint(new_ordered_dict_location,indent=1)

#for 3 locations 1m33s

In [23]:
#for person
person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
cat_with_pv_person = {}

#print(person_questions_dict)
#del str

#for person
cat_ranking_person = get_categories(person_questions_dict)
#pprint.pprint(cat_ranking_person, indent=1)
cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
#pprint.pprint(cat_with_pv_person, indent=1)
#for person: sorting the dict after pages per category
categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
#pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)
#pprint.pprint(new_ordered_dict_person,indent=1)

#for 3 people 17m22s
#27s
# for 20 F1 drivers -> 20mins

Sorting failed for location Max Verstappen: tuple index out of range
Sorting failed for location Michael Jackson: tuple index out of range


In [24]:
#just for easy printing such that you dont have to0 calculate everything everytime
copy_categories_with_subs_and_pageviews_location = categories_with_subs_and_pageviews_location
copy_new_ordered_dict_location = new_ordered_dict_location
#print("Ordered after the number of pages per category:")
#pprint.pprint(copy_categories_with_subs_and_pageviews_location,indent=1)
#print("Ordered after the number of pageviews per category:")
#pprint.pprint(copy_new_ordered_dict_location,indent=1, compact=True)

copy_categories_with_subs_and_pageviews_person = categories_with_subs_and_pageviews_person
copy_new_ordered_dict_person = new_ordered_dict_person
#print("Ordered after the number of pages per category:")
#pprint.pprint(copy_categories_with_subs_and_pageviews_person,indent=1)
#print("Ordered after the number of pageviews per category:")
#prune the dict down to 20 most visited categories per person
#copy_new_ordered_dict_person = prune_ordered_dict(copy_new_ordered_dict_person, 20)
#first delete categories like (livingpeople, deaths, births) and prune the dict down to 20 most visited categories per person
copy_new_ordered_dict_person_test = prune_and_ordered_dict(copy_new_ordered_dict_person, 20)
copy_new_ordered_dict_person = sorting_dict(copy_new_ordered_dict_person_test)
#pprint.pprint(copy_new_ordered_dict_person,indent=1, compact=True)

number_categories_per_person = {}
for key, value in copy_new_ordered_dict_person.items():
    number_categories_per_person[key] = len(value)

#8.30 for 10 drivers
category_occurences = find_most_common_links(copy_new_ordered_dict_person)
#print('These are the categories that occur most between the person-entities:')
#pprint.pprint(category_occurences, indent=1, compact=True)

Sorting failed for location Max Verstappen: tuple index out of range


## Functions for the unexpected-categories approach:

In [48]:

"""
1. retrieve all of the related links of a wiki-persons-page (first 500);
2. pre prune the list, such that only entries with 2 words in the title are left; (for performance reasons we assume 2 words equaly is human)
3. analyse the links that are left via wikipedia APi and the "instance of" (P31) property set to "human" (Q5)
4. now we have a list of related people to compare to our answer-person-entity
5.1 retrieve the categories of the answer_entities (top 10-20)
5.2 retrieve the categories of those related people as done above with the answer_entities (top 10-20)

6. compare the categories of the related-people of the corresponding answer-entity and list the ones that appear most often (between answer-entity=A AND related_people of A )
7. look for the "unexpected", a popular category that is well known and where from a list of related_people our answer-entity is the only one that appears in

8. create some metrics and threshholds -> testing
"""

"""
Function that retrieves the related/popular people of the answer-entities;
Args:     the answer-entity name 
Returns:  a list where each entry of the list is a dictionary; the keys and values are the (title and url) of the related people
"""
def get_related_people_from_person_name(person_name):
  wiki_link = 'https://en.wikipedia.org/wiki'
  related_people_list = []

  modified_text = person_name.replace(' ', '_') #replace spaces with underscores in the name to use it in link
  link = f"{wiki_link}/{modified_text}" 
  #print(link)
  related_articles = get_related_links(link)
  filtered_links = filter_two_word_titles(related_articles)
  related_people_list = check_if_person(filtered_links)
  
  return related_people_list

"""
Function that gets the pageviews of up to 10 links at a time;
Args:     dictionary where the keys are the answer-entities and the value is a link_list 
Returns:  dictionary where the keys are the answer-entities and the value is a list with the links and pageviews
"""
def get_pageviews_from_links(link_dict):
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=last-year&pages='
  return_dict = {}

  for key,value in link_dict.items():
    link_list = value
    
    pruned_link_parts_list = extract_last_parts(link_list)
    concat_str_for_links = concatenate_elements(pruned_link_parts_list)             #combine up to 10 of these links to create a request to pageview
    pageviews_url_list = combine_pv_urls(pageviews_range_url, concat_str_for_links) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page (REUSED FROM YEARS PART)
    categories_with_pageviews =combine_dicts_from_links(pageviews_url_list)         #now we called the links and retreieved the pageviews; saved them as a dictionary
    ordered_categories_with_pageviews = sort_dict_desc(categories_with_pageviews)   #now the list is ordered in ascending order
    
    return_dict[key] = ordered_categories_with_pageviews
  
  return return_dict

"""
Function that takes the 5 most popular related-people from the related_people_pageviews_dict and  
retrieves the corresponding wiki-categories with the their pageviews;
Args:     dictionary where the keys are the answer-entities and the value is a dict of related people with their pageviews 
Returns:  dictionary where the keys are the answer-entities and the value is 
  a dict where the keys are the related-people and the valus are tupples with the 10 most popular categories they appear and with the pageviews of that category
"""
def get_categories_of_people_list(people_list, limit=5):
  return_dict = {}
  #intermediate_relatedpeople_with_categories_dict = {}

  for key,value in people_list.items():
    related_people_orderd = dict(sorted(value.items(), key=lambda x: x[1], reverse=True))   #order the dict after the pageviews in descending order
    top_most_popular_people = dict(itertools.islice(related_people_orderd.items(), 5))      #take the top 5 most known people from the list

    if len(related_people_orderd) == 0 or  len(top_most_popular_people) == 0:
      continue

    categories_of_related_people = get_categories(top_most_popular_people)
    #print("Test_1")
    categories_with_pageviews_person = get_pageviews_for_categories(categories_of_related_people)
    #print("Test_2")
    categories_with_subs_and_pageviews_person = get_dict_for_every_location(categories_of_related_people, categories_with_pageviews_person)

    #print("Test_3")
    new_ordered_dict_related_person = sorting_dict(categories_with_subs_and_pageviews_person)
    copy_new_ordered_dict_person_test = prune_and_ordered_dict(new_ordered_dict_related_person, 10)   
    ordered_dict_related_person = sorting_dict(copy_new_ordered_dict_person_test)
    return_dict[key] = ordered_dict_related_person
  return return_dict

"""
Function that is created with help of reusable function from above
Args:     person_question_dict with all people with their questions 
Returns:  dictionary where the keys are the answer-entities and the value is a OrderedDict of all categories with pageviews
"""
def get_categories_with_pv_answerEntities():
  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
  cat_ranking_person = get_categories(person_questions_dict)
  #pprint.pprint(cat_ranking_person, indent=1)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  #pprint.pprint(cat_with_pv_person, indent=1)
  #for person: sorting the dict after pages per category
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  #pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)

  return new_ordered_dict_person

"""
Counts the occurrences of categories (links) in the input dictionary.
Args: data (dict): A dictionary containing category links as keys and tuples as values.
Returns: dict: A new dictionary where the keys are the category links and the values are
  the corresponding tuples of the category with a middle value indicating how
  often the link/category appeared in the other keys.
"""
def count_categories(related_people_with_categories, answer_entities_with_categories):
  category_appereances = {}

  for answerEntityKey, aeCategory in answer_entities_with_categories.items():
    inner_dict = {}
    for answerKey, relatedDict in related_people_with_categories.items():
      if answerEntityKey == answerKey:
        for catLink, tupl in aeCategory.items():
          people_list = []
          for relatedPersonKey, catWithPvs in relatedDict.items():
            for relatedLink, relatedTupl in  catWithPvs.items():
              if catLink == relatedLink:
                rel_pers_str = str(relatedPersonKey)
                if rel_pers_str not in people_list and catLink == relatedLink:
                  people_list.append(rel_pers_str)
                inner_dict[relatedLink] = (len(people_list),relatedTupl, people_list)
    category_appereances[answerEntityKey] = inner_dict

  return category_appereances

"""
Calculates the Intersection over Union between the answer-entity and each of the related-person entries seperately
Args: data (dict): A dictionary containing the answer-entity with the corresponding related-people and their most popular categories that they share with ther answer-entity
Returns: dict: A dict where the value is a list of tuple like this: ('Max Verstappen', 'Charles Leclerc', 5, 20, 0.25) (person_a, person_b, num_shared_categories, num_total_categories, num_shared_categories/num_total_categories)
"""
def calculate_IoU_from_countedCategoryDict(counted_category_apperances):
  #first we recover the list of people that are related to answer-entity
  p_dict = {}
  for key,value in counted_category_apperances.items():
    person_list = []
    for link, f_tup in value.items():
      #print(f_tup)
      num= f_tup[0]
      tup= f_tup[1] 
      p_lst = f_tup[2]
      for person in p_lst:
        if person not in person_list:
          person_list.append(person)
    p_dict[key] = person_list
  
  iou_between_person_list = {}
  
  #now calculate the IoU
  for key_cat,value_cat in counted_category_apperances.items():
    for key,value in p_dict.items():
      if key == key_cat:
        inter_list = {}
        for person in value:
          p_count = 0
          for link, f_tup in value_cat.items():
            p_lst = f_tup[2]
            if person in p_lst:
              p_count += 1
          inter_list[person] = p_count
      iou_between_person_list[key_cat] = inter_list

  IoU_dict = {}
  for key, value in iou_between_person_list.items():
    IoU_list = []
    for person, number in value.items():
      num_total_categs = 20
      IoU_list.append((person,number,num_total_categs, number/num_total_categs))
    IoU_dict[key] = IoU_list

  #return iou_between_person_list
  return IoU_dict


"""
Calculates the avg_diversity_from_IoU
Args: 
Returns: 
"""
def calculate_avg_diversity_from_IoU(intersection_between_people_with_ae):
  avg_diversity_dict = {}

  for key,value in intersection_between_people_with_ae.items():
    diversity_sum = 0
    pairwise_comparisons = 0
    for item in value:
      all_categs = item[2]
      IoU = item[1]
      diversity_sum += (all_categs - IoU)
      pairwise_comparisons += 1
    avg_diversity_dict[key] = (diversity_sum / pairwise_comparisons)
  return avg_diversity_dict


"""
Calculates the categories score from the category diversity (calculated by calculate_avg_diversity_from_IoU() ) and the cat_popularity (=pageviews)
Args: 
Returns: 
"""
def calculate_categories_score(counted_category_apperances, avg_diversity_from_IoU):
  categories_scores_dict = {}
  for key,value in counted_category_apperances.items():
    inter_dict={}
    for item in value.items():
      link = item[0]
      cat_popularity = item[1][1][2]
      for name, cat_div in avg_diversity_from_IoU.items():
        if name == key:
          inter_dict[link] = cat_popularity * cat_div
    categories_scores_dict[key] = inter_dict
  
  ordered_dicter = {}
  ordered_scores ={}
  for k,v in categories_scores_dict.items():
    ordered_scores[k] = OrderedDict(v)
  for k,v in ordered_scores.items():
    ordered_dicter[k] = OrderedDict(sorted(v.items(), key=lambda x: x[1], reverse=True))
  
  #modify the scores to give some categories a lower one
  for key,value in ordered_dicter.items():
    for link, score in value.items():
      if '20th' in link or '21st' in link:
        value[link]  = score / 4
  
  return ordered_dicter


template_sentence_person_list = ['The person you are looking for is/was occupied as 0 and a member of the category 1', 'The person you are looking for is/was active in the occupation 0 and appears in the category: 1']
#takes the categories scores dict and chooses the category with the highest score
def create_hint_sentences_unexCategs(categories_scores_dict):
  people_occupations = get_occupations(test_person_answers_dict)
  most_unexpected_categories_dict = {}
  hint_sentence_unexCateg_dict = {}

  for key,value in categories_scores_dict.items():
    categories_scores_dict[key] = OrderedDict(sorted(value.items(), key=lambda x: x[1], reverse=True))
    most_unexpected_categories_dict[key] = (next(iter(value.items())), people_occupations[key])
    
  for key,value in most_unexpected_categories_dict.items():
    hint_sentence_unexCateg_dict[key] = []
    if people_occupations[key][0] == 'Racing':
      occu_str = people_occupations[key][0] + people_occupations[key][1]
    else:
      occu_str = people_occupations[key][0]
    for sentence in template_sentence_person_list:
      hint_sentence_unexCateg_dict[key].append( sentence.replace('0', occu_str).replace('1', get_category_title(most_unexpected_categories_dict[key][0][0]).split(':')[-1].replace('_', ' ') ))

  return hint_sentence_unexCateg_dict

In [42]:
#CREATE A FUNCTION THAT PUTS EVERYTHING TOGETHER - FOR UNEXPECTED CATEGORIES
"""
1. retrieve all of the related links of a wiki-persons-page (first 500);
2. pre prune the list, such that only entries with 2 words in the title are left; (for performance reasons)
3. analyse the links that are left via wikipedia APi and the "instance of" (P31) property set to "human" (Q5)
4. now we have a list of related people to compare to our answer-person-entity
5. search for the unexpected category: a category that is popular/well-known, but the answer-entity is one of the only entries from his occupation that appears in the category
6. rank them via the calculated scores
7. create hint-sentences with it
"""
def get_person_hints_unexpected_categories():
  person_answers_dict = dict(zip(person_df['Answer'], person_df['Question']))
  related_people_dict = {}
  related_people_link_dict= {}
  related_people_pageviews_dict = {}
  most_popular_related_people_with_categories = {}

  #time saving for first part (related people recovery) 2m
  for key,value in person_answers_dict.items():
    related_people_dict[key] = get_related_people_from_person_name(key)
  
  #time saving for second part (related peoples with pageviews and ordering) - 16m (6-7m)
  for key,value in related_people_dict.items():
    inter_link_list = []
    for item in value:
      inter_link_list.append(item['url'])
    related_people_link_dict[key] = inter_link_list
    related_people_pageviews_dict = get_pageviews_from_links(related_people_link_dict)
  #pprint.pprint("test_related_people_pageviews_dict" )
  #pprint.pprint( related_people_pageviews_dict)


  #time saving for third part (related peoples categories recovery and ordering) - 43m+ (14m-24m)
  most_popular_related_people_with_categories = get_categories_of_people_list(related_people_pageviews_dict)
  pprint.pprint("most_popular_related_people_with_categories" )
  pprint.pprint(most_popular_related_people_with_categories)

  #time saving for third part retrieves the categories of the answer-entities - 9m+ (6m)
  answer_entities_with_categories = get_categories_with_pv_answerEntities()
  #pprint.pprint(test_answer_entities_with_categories, indent=1, compact=True)

  #time saving for fourth part counts the categories of the answer-entities - 3s
  counted_category_apperances = count_categories(most_popular_related_people_with_categories, answer_entities_with_categories)
  #just for the ordering of the inner list
  ordered_data = {}
  for key,value in counted_category_apperances.items():
    tmp = OrderedDict(value)
    ordered_data[key] = OrderedDict(sorted(tmp.items(), key=lambda x: x[1][0], reverse=True) )
  counted_category_apperances = ordered_data
  pprint.pprint(counted_category_apperances)


  #1. calculate the IoU between max and every other person - (M,C) = 5/20; (M,L) = 2/20; (M,D) = 2/20; (M,A) = 2/20; (M,F) = 2/20;
  intersection_between_people_with_ae = calculate_IoU_from_countedCategoryDict(counted_category_apperances)
  #2. calculate the average diversity between the 6 drivers - (20-5) + (20-2) + (20-2) + (20-2) + (20-2) = 87; (#avg_diversity/#pairwise_comparison) = 87/5 = 17,4
  avg_diversity_from_IoU = calculate_avg_diversity_from_IoU(intersection_between_people_with_ae)
  #3. calculate a type of categoreis_score - categories_score = cat_diversity * cat_popularity(pvs)
  categories_scores_dict = calculate_categories_score(counted_category_apperances, avg_diversity_from_IoU)
  for key,value in categories_scores_dict.items():
    categories_scores_dict[key] = OrderedDict(sorted(value.items(), key=lambda x: x[1], reverse=True))
  #create some sentences with the occupation and a unexpected category as hints
  mucd = create_hint_sentences_unexCategs(categories_scores_dict)

  return mucd

In [67]:
#ca. 25m of execution for 2 people
unexpected_categories_hints = get_person_hints_unexpected_categories()
pprint.pprint(unexpected_categories_hints, indent=2)

KeyboardInterrupt: ignored

#### The functions and function-calls below were mainly used to create the get_person_hints_unexpected_categories function

In [51]:
"""
1. retrieve all of the related links of a wiki-persons-page (first 500);
2. pre prune the list, such that only entries with 2 words in the title are left; (for performance reasons)
3. analyse the links that are left via wikipedia APi and the "instance of" (P31) property set to "human" (Q5)
4. now we have a list of related people to compare to our answer-person-entity
"""
#this is just for reducing time during testing - 7m+ (30s-4m)

test_person_answers_dict = dict(zip(person_df['Answer'], person_df['Question']))
test_related_people_dict = {}
test_related_people_link_dict= {}
test_related_people_pageviews_dict = {}
test_most_popular_related_people_with_categories = {}

#time saving for first part (related people recovery) 2m
for key,value in test_person_answers_dict.items():
  test_related_people_dict[key] = get_related_people_from_person_name(key)
#pprint.pprint(test_related_people_dict)

In [52]:
#time saving for second part (related peoples with pageviews and ordering) - 16m (6-7m)
for key,value in test_related_people_dict.items():
  inter_link_list = []
  for item in value:
    inter_link_list.append(item['url'])
  test_related_people_link_dict[key] = inter_link_list
  test_related_people_pageviews_dict = get_pageviews_from_links(test_related_people_link_dict)
#pprint.pprint(test_related_people_pageviews_dict)

In [58]:
for key,value in test_related_people_pageviews_dict.items():
  related_people_orderd = dict(sorted(value.items(), key=lambda x: x[1], reverse=True))   #order the dict after the pageviews in descending order
  top_most_popular_people = dict(itertools.islice(related_people_orderd.items(), 5))

  pprint.pprint(related_people_orderd)
  pprint.pprint(top_most_popular_people)

  categories_of_related_people = get_categories(top_most_popular_people)
  pprint.pprint(categories_of_related_people)
  print("\n") #PAAAAASST

  categories_with_pageviews_person = get_pageviews_for_categories(categories_of_related_people)
  pprint.pprint(categories_with_pageviews_person)
  print("\n")
  
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(categories_of_related_people, categories_with_pageviews_person)
  pprint.pprint(categories_with_subs_and_pageviews_person)
  print("\n")



Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
                                                                                  'Julie '
                                                                                  'Grant',
                                                                                  'Larry '
                                                                                  'Grayson',
                                                                                  'Anita '
                                                                                  'Harris',
                                                                                  'Ronnie '
                                                                                  'Hawkins',
                                                                                  'Colin '
                                                                                  'Hicks '
                 

In [59]:
#time saving for third part (related peoples categories recovery and ordering) - 43m+ (14m-24m)
test_most_popular_related_people_with_categories = get_categories_of_people_list(test_related_people_pageviews_dict)
pprint.pprint(test_most_popular_related_people_with_categories)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
                                                                                  'Nash',
                                                                                  'Kim '
                                                                                  'Newman',
                                                                                  'John '
                                                                                  'Nightingale '
                                                                                  '(police '
                                                                                  'officer)',
                                                                                  'Susan '
                                                                                  'Nye, '
                                                                                  'Baroness '
             

In [60]:
#time saving for third part retrieves the categories of the answer-entities - 9m+ (6m)
test_answer_entities_with_categories = get_categories_with_pv_answerEntities()
#pprint.pprint(test_answer_entities_with_categories, indent=1, compact=True)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
                                                                                                                     'Lexington '
                                                                                                                     'Avenue '
                                                                                                                     'Line)',
                                                                                                                     '35th '
                                                                                                                     'Virgin '
                                                                                                                     'Islands '
                                                                                                                     'Legislature',
                                      

In [61]:
#time saving for fourth part counts the categories of the answer-entities - 3s
test_counted_category_apperances = count_categories(test_most_popular_related_people_with_categories, test_answer_entities_with_categories)
#just for the ordering of the inner list
ordered_data = {}
for key,value in test_counted_category_apperances.items():
  tmp = OrderedDict(value)
  ordered_data[key] = OrderedDict(sorted(tmp.items(), key=lambda x: x[1][0], reverse=True) )
test_counted_category_apperances = ordered_data
#pprint.pprint(test_counted_category_apperances)

In [62]:
#1. calculate the IoU between max and every other person:
# (M,C) = 5/20; (M,L) = 2/20; (M,D) = 2/20; (M,A) = 2/20; (M,F) = 2/20;
intersection_between_people_with_ae = calculate_IoU_from_countedCategoryDict(test_counted_category_apperances)
# print("IoU between the people and the answer-entity:")
# pprint.pprint(intersection_between_people_with_ae,indent=4)

#2. calculate the average diversity between the 6 drivers:
# (20-5) + (20-2) + (20-2) + (20-2) + (20-2) = 87; (#avg_diversity/#pairwise_comparison) = 87/5 = 17,4
avg_diversity_from_IoU = calculate_avg_diversity_from_IoU(intersection_between_people_with_ae)
# print("Avergage diversity from IoU:")
# pprint.pprint(avg_diversity_from_IoU, indent=4)

#3. calculate a type of categoreis_score
# categories_score = cat_diversity * cat_popularity(pvs)
categories_scores_dict = calculate_categories_score(test_counted_category_apperances, avg_diversity_from_IoU)
# print("Category score:")
for key,value in categories_scores_dict.items():
  categories_scores_dict[key] = OrderedDict(sorted(value.items(), key=lambda x: x[1], reverse=True))
  #ordered_dicter[k] = OrderedDict(sorted(v.items(), key=lambda x: x[1], reverse=True))
# pprint.pprint(categories_scores_dict,indent=4)

# retrieve the occupation of our answer-entities for better hint sentence creation
#people_occupations = get_occupations(test_person_answers_dict)
#pprint.pprint(people_occupations,indent=4)

#create some sentences with the occupation and a unexpected category as hints
mucd = create_hint_sentences_unexCategs(categories_scores_dict)
# pprint.pprint(mucd)

#now we could try and compare the person-question and the hints via BERNT and choose the most similar one

 For Testing and research: Execute this part once (ca. 60m for 3people), then ruese the test variables from this section (faster)

#### Continue testing

In [ ]:
#Testing new things and approaches
wiki_link = 'https://en.wikipedia.org/wiki/'

def check_inner_tuple_length(pv_and_pages_dict):
  # Iterate over the dictionary
  for key, value in pv_and_pages_dict.items():
    # Iterate over the ordered dict values
    for inner_key, inner_value in value.items():
      # Check if the inner tuple has two elements
      if len(inner_value) == 2:
        # Update the value by adding a zero at the end
        value[inner_key] = inner_value + (0,)
  return pv_and_pages_dict

def unexpected_categories(person_answer_entity_list):
  wiki_link_list={}
  for val in person_answer_entity_list:
    names_underscores = val.replace(' ', '_')
    answer_entity_wiki_link = wiki_link + names_underscores
    wiki_link_list[val] =  answer_entity_wiki_link
  
  categories = get_categories(wiki_link_list)
  categories_with_pv = get_pageviews_for_categories(categories)
  categories_with_subs_pvs = get_dict_for_every_location(categories, categories_with_pv)

  test1 = check_inner_tuple_length(categories_with_subs_pvs )
  test2 = prune_and_ordered_dict(test1, 20)
  ordered_categories_dict_AnswerPeopleEntities = sorting_dict(test2)
  #up to this point, the function retrieves,pre-prunes and orderes the most popular categories from the person-answer-entities
  #now wee need to do the same for each of the selected_related_people_list, to then compare the categories and rank them from most to least occuring
  return(ordered_categories_dict_AnswerPeopleEntities)

def related_people_with_categories(categs_list):
  selected_people = {}
  selected_people_with_categories = {}
  for key, value in categs_list.items():
    names_underscores = key.replace(' ', '_')
    related_articles = get_related_links(wiki_link + names_underscores)
    filtered_links = filter_two_word_titles(related_articles)
    related_people_list = check_if_person(filtered_links)
    select_random_people = select_random_entries(related_people_list)

    selected_people[key] = select_random_people

  return selected_people


In [ ]:
"""
TEST OF FUNCTION FROM THE CELLS ABOVE
continue here
Loops and function-calls to get the most popular related people of each answer-entity,
with the categories of each related-person ranked from most-popular to least,
to then beeing able of comparing those categories and finding an unusual one, for this specific answer-entity;
"""

#Testing out the new functions: 
person_answers_dict = dict(zip(person_df['Answer'], person_df['Question']))
related_people_dict = {}
related_people_link_dict= {}
related_people_pageviews_dict = {}

#PART 1:
#for key,value in person_answers_dict.items():
#  related_people_dict[key] = get_related_people_from_person_name(key)

#this is just for reducing time during testing 
related_people_dict = test_related_people_dict
#pprint.pprint(related_people_dict)

#PART 2:
# for key,value in related_people_dict.items():
#   inter_link_list = []
#   for item in value:
#     inter_link_list.append(item['url'])
#   related_people_link_dict[key] = inter_link_list
#   related_people_pageviews_dict = get_pageviews_from_links(related_people_link_dict)

#this is just for reducing time during testing 
related_people_pageviews_dict = test_related_people_pageviews_dict
#pprint.pprint(related_people_pageviews_dict)

#PART 3:
#most_popular_related_people_with_categories = get_categories_of_people_list(related_people_pageviews_dict)
most_popular_related_people_with_categories = test_most_popular_related_people_with_categories
#pprint.pprint(most_popular_related_people_with_categories)

#PART 4:
#creates a list with all the answer-entity names
#answer_entities_with_categories = get_categories_with_pv_answerEntities()
answer_entities_with_categories = test_answer_entities_with_categories
#pprint.pprint(answer_entities_with_categories, indent=1, compact=True)

#PART 4:
#compare anc count
#counted_category_apperances = count_categories(most_popular_related_people_with_categories, answer_entities_with_categories)
counted_category_apperances = test_counted_category_apperances
# pprint.pprint(counted_category_apperances, indent=1, compact=True)

#shared_categories_between_people = count_categories(most_popular_related_people_with_categories)
#pprint.pprint(shared_categories_between_people, indent=1, compact=True)

#Now compare the categories: 
#Most appearing category between person-answer-entity and the corresponding related-people
#Most popular category, that corresponds to just the person-answer-entity and as few of the related-people as possible (=unexpected category)



# now calculate the most popukar categories of the selected_people and compare them

person_answer_entity_list = list(person_df['Answer'])
#print(person_answer_entity_list)
#unexpected_categories(person_answer_entity_list)
#print(unexpected_categories(person_answer_entity_list))
a = unexpected_categories(person_answer_entity_list)
# pprint.pprint(a )
#pprint.pprint(unexpected_categories(person_answer_entity_list))
#b = related_people_with_categories(a)
#pprint.pprint(b)
#print(person_questions_dict)
#10 mins

## Functions for the unexpected-predicate approach:

Now, the above was about category membership, but how about predicates. 

Again compare the predicates of our answer-entity with the predicates of similar people (calculated above in related_people) and find a special/unexpected/surprising predicate about our answer-entity.

By similar people here we could take ones that we have already calculated in the unexpected-categories approach.

---

Say our John Pack has 5 predicates such as: 
*   p1 - he has 2 kids, 
*   p2 - he was married 5 times, 
*   p3 - he is 170cm tall, 
*   p4 - he was born in White Stream, Alabama,
*   p5 - he was in prison when in his 20s. 

Which one is good to use as a hint? 
I believe the one that is exceptional, hence, based on that thinking p1, p3 are not good, p2, p4, p5 are good. 

What scoring strategy would select p2, p4, p5? 
I believe one based on how often similar people like our John have the values of same predicates. That is, how many US politicians have more than 2 wives in their life? how many have kids, 2 kids, etc. 

In [31]:
#FUNCTIONS HERE
#https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples#Subproperties_of_location_(P276)
# Subproperties of location (P276)
# All properties with descriptions and aliases and types
# MV link:https://www.wikidata.org/wiki/Q2239218


import requests

def get_wikidata_entries(wikidata_link):
    try:
        # Fetch the statements for the Wikidata item
        statements_url = f"{wikidata_link}?action=wbgetclaims&format=json"
        response = requests.get(statements_url)
        statements_data = response.json()

        # Extract the desired entries from the statements
        entries = {}

        # Instance of
        if "P31" in statements_data["claims"]:
            entries["instance of"] = statements_data["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"]["label"]

        # Image
        if "P18" in statements_data["claims"]:
            image_url = statements_data["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
            entries["image"] = image_url.split("/")[-1]

        # Sex or gender
        if "P21" in statements_data["claims"]:
            entries["sex or gender"] = statements_data["claims"]["P21"][0]["mainsnak"]["datavalue"]["value"]["label"]

        # Country of citizenship
        if "P27" in statements_data["claims"]:
            country_of_citizenship = []
            for claim in statements_data["claims"]["P27"]:
                country_of_citizenship.append(claim["mainsnak"]["datavalue"]["value"]["label"])
            entries["country of citizenship"] = country_of_citizenship

        return entries

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data: {e}")
        return {}


import requests

def get_predicates_for_person(person_name):
    try:
        # Query the Wikidata API to retrieve the person's Wikidata ID
        search_url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&format=json&language=en&type=human&search={person_name}"
        response = requests.get(search_url)
        search_data = response.json()

        if search_data.get("search"):
            wikidata_id = search_data["search"][0]["id"]
        else:
            print(f"No Wikidata ID found for '{person_name}'")
            return {}

        # Query the Wikidata API to retrieve the predicates for the person's Wikidata ID
        predicates_url = f"https://www.wikidata.org/w/api.php?action=wbgetclaims&format=json&entity={wikidata_id}"
        response = requests.get(predicates_url)
        predicates_data = response.json()

        # Extract the predicates and their values
        predicates = {}
        for predicate_id, predicate_info in predicates_data["claims"].items():
            predicate_label = predicate_info[0]["mainsnak"]["property"]
            predicate_value = predicate_info[0]["mainsnak"]["datavalue"]["value"]
            predicates[predicate_label] = predicate_value

        return predicates

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data: {e}")
        return {}



In [35]:
#Test here
#start here with predicates
page_title = "https://en.wikipedia.org/wiki/Max_Verstappen"
#test_predicates = get_predicates_from_wikipedia_page(page_title)
#pprint.pprint(test_predicates)

pprint.pprint(get_predicates_for_person('Max_Verstappen'))

No Wikidata ID found for 'Max_Verstappen'
{}


In [33]:
#CREATE A FUNCTION THAT PUTS EVERYTHING TOGETHER - FOR UNEXPECTED PREDICATES
"""
1. retrieve all of the related links of a wiki-persons-page (first 500);
2. pre prune the list, such that only entries with 2 words in the title are left; (for performance reasons)
3. analyse the links that are left via wikipedia APi and the "instance of" (P31) property set to "human" (Q5)
4. now we have a list of related people to compare to our answer-person-entity

5. 
5. search for the unexpected category: a category that is popular/well-known, but the answer-entity is one of the only entries from his occupation that appears in the category
6. rank them via the calculated scores
7. create hint-sentences with it
"""
def get_person_hints_unexpected_predicates():
  print("Hallo")

# **Old predictions methods**

## Functions for reuse

In [ ]:
#get Wikidata ID
def getQitemOfName(entity):
  
  searched = "'"+entity+"'"
  sparql.setQuery('''
  SELECT ?item ?itemLabel WHERE {
    ?item rdfs:label '''+searched+'''@en.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }

  }
  ''')
  
  sparql.setReturnFormat(JSON)
  entities = sparql.query().convert()
  entities_df = pd.json_normalize(entities['results']['bindings'])
  words = entities_df[["item.value"]].iloc[0].str.split("/")[0]
  return words[-1]

In [ ]:
#get name of entity from Wikidata ID

def getLabelOfQitems(entities):

  finalLabels = []
  for item in entities:
    #searched = "'"+item+"'"
    sparql.setQuery('''
    SELECT * WHERE {
        wd:'''+item+ ''' rdfs:label ?label .
        FILTER (langMatches( lang(?label), "EN" ) )
    } LIMIT 1
    ''')
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])
    label = entities_df["label.value"].iloc[0]
    finalLabels.append(label)

  return finalLabels  

In [ ]:
#function to get all backlinks to wikipage and saves backlinks >= 1000 in array
#source: https://www.mediawiki.org/wiki/API:Backlinks#Response

import requests

def getBacklinks(entities):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  listWithBacklink500 = []
  dictWithAllEntries = {}

  for val in entities:
    #print(row["itemLabel.value"])
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": val,
        "bllimit": "max"
    }

    try:
      R = S.get(url=URL, params=PARAMS)
      DATA = R.json()
   

      BACKLINKS = DATA["query"]["backlinks"]

      count = 0
      for links in BACKLINKS:
        count +=1

      
      while "continue" in DATA:
        if (count >= 1000):
          break;
        blcontinue = DATA["continue"]["blcontinue"]
        PARAMS["blcontinue"] = blcontinue

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        BACKLINKS = DATA["query"]["backlinks"]

        for links in BACKLINKS:
          count += 1
      
      dictWithAllEntries[val] = count
      #print(val + ": " + str(count))

    except ValueError:
      print("Value Error")

    if (count >= 1000):
      listWithBacklink500.append(val) 

  if (len(listWithBacklink500) > 0):
    #return getPageViews(listWithBacklink500)
    return listWithBacklink500

  else:
    listMostBacklinks = []
    sortedDict = dict(sorted(dictWithAllEntries.items(), key=lambda item:item[1]))

    for k, v in sortedDict.items():
      listMostBacklinks.append(k)
    
    numberToPass = len(listMostBacklinks)//3
        
    return listMostBacklinks[-numberToPass:]

In [ ]:
#function to get all backlinks to wikipage and saves backlink == 500 in array
#source: https://www.mediawiki.org/wiki/API:Backlinks#Response

import requests

def getBacklinksDict(entities):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  dictWithBacklink500 = {}
  dictWithAllEntries = {}

  for key, val in entities.items():
    print(key, val)

    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": key,
        "bllimit": "max"
    }

    try:
      R = S.get(url=URL, params=PARAMS)
      DATA = R.json()
   

      BACKLINKS = DATA["query"]["backlinks"]

      count = 0
      for links in BACKLINKS:
        count +=1

      
      while "continue" in DATA:
        if (count >= 1000):
          break;
        blcontinue = DATA["continue"]["blcontinue"]
        PARAMS["blcontinue"] = blcontinue

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        BACKLINKS = DATA["query"]["backlinks"]

        for links in BACKLINKS:
          count += 1
      
      dictWithAllEntries[key] = count
      #print(val + ": " + str(count))

    except ValueError:
      print("Value Error")
    except KeyError:
      print("Key Error")

    if (count >= 1000):
      dictWithBacklink500.update({key: val}) 

  if (len(dictWithBacklink500) > 0):
    #return getPageViews(listWithBacklink500)
    return dictWithBacklink500

    #print(listMostBacklinks[-numberToPass:])

    
    return listMostBacklinks[-numberToPass:]

In [ ]:
#get avg pageViews of entities

def getPageViewsList(listEntities):
  avgDict = {}
  
  for val in listEntities:
    try:
      resp = pageviewapi.per_article('en.wikipedia', val, '20191106', '20201120', access='all-access', agent='all-agents', granularity='monthly')
    #print(resp)
      avgViews = 0
      count = 0
      for i in resp.get("items"):
        #print(i.get("views"))
        avgViews += i.get("views")
        count += 1
      avgDict.update({val: avgViews//count})
    except:
      print("Page not found to check pageViews for page: " + val)
      
  #return (max(avgDict, key=avgDict.get))
  return avgDict


  


In [ ]:
#get avg pageViews of entity
import pageviewapi


def getPageViews(entity):
  avgDict = {}
  
  try:
    resp = pageviewapi.per_article('en.wikipedia', entity, '20191106', '20201120', access='all-access', agent='all-agents', granularity='monthly')

    avgViews = 0
    count = 0
    for i in resp.get("items"):
      #print(i.get("views"))
      avgViews += i.get("views")
      count += 1

    pageViews = avgViews//count
    #avgDict.update({val: avgViews//count})

    #return (max(avgDict, key=avgDict.get))
    return pageViews
  #print(resp)
  except:
    print("Page not found to check pageViews for page: " + entity)
    return -1
    


In [ ]:
#check if entity is a person
#return list of persons

def checkIfHuman(entity):

  #listHumans = []

  #for test in entity:
    #print(item)
  searched = "'"+entity+"'"

  #print(searched)

  try:

    sparql.setQuery('''
        SELECT distinct ?item ?itemLabel 
        WHERE{
          ?item ?label '''+searched+'''.  
          ?item wdt:P31 wd:Q5.
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.	
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
        }
    ''')
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])

    #print(entities)
    #print(entities_df["itemLabel.value"].item())

    #for index, row in entities_df.iterrows():
    return entities_df["itemLabel.value"].item()
    
    
  except:
    return None


In [ ]:
#if parantheses in entity -> remove

def removeParanthesesDict(inputDict):

  newDict = {}
  for key, val in inputDict.items():
    if ('(' in key):
      newElem = re.sub("[\(\[].*?[\)\]]", "", key).rstrip()
      newDict.update({newElem: val})
    else:
      newDict.update({key: val})

  return newDict

##Hints if answer is a Location

In [ ]:
#Query with all properties to get data

def allHintsLocation(entity):
  sparql.setQuery('''
  SELECT ?itemLabel ?property
  WHERE
  {  
    VALUES ?property {wdt:P35 wdt:P30 wdt:P36 wdt:P610 wdt:P1082 wdt:P421 wdt:P38 wdt:P17 wdt:P1376 wdt:P131 wdt:P6 wdt:P1830 wdt:P47 wdt:P206 wdt:P37 wdt:P463 wdt:793}
    wd:'''+entity+''' ?property ?item.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  hintsLocation = sparql.query().convert()

  hintsLocation_df = pd.json_normalize(hintsLocation['results']['bindings'])

  return hintsLocation_df

In [ ]:
#create template based hints

def getFormatedHintsLocation(queryResults_df):
  timeZone = []
  isLocatedIn = []
  ownerOf = []
  shareBorder = []
  bodyOfWater = []
  languages = []
  memberOf = []
  significantEvents = []

  listHintsLocation = []

  searchString = "http://www.wikidata.org/prop/direct/"
  
  for index, row in queryResults_df.iterrows():
    if(not location in row["itemLabel.value"]):
      if(row["property.value"] == searchString + "P35"):
        listHintsLocation.append("Head of state of searched country is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P30"):
        listHintsLocation.append("The searched location is on continent " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P36"):
        listHintsLocation.append("The capital of searched country is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + ""):
        listHintsLocation.append("The highest point in searched location is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P1082"):
        listHintsLocation.append("The searched location has a population of " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P38"):
        listHintsLocation.append("Currency in searched location is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P17"):
        if(not location in row["itemLabel.value"]):
          listHintsLocation.append("The searched location is in country "+ row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P1376"):
        listHintsLocation.append("The searched city is capital of " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P131"):
        listHintsLocation.append("The searched location is located in " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P6"):
        listHintsLocation.append("Head of government of searched location is " + row["itemLabel.value"])
      
      #if get multiple values save all in list and pick one afterwards
      elif(row["property.value"] == searchString + "P1830"):
        if(not location in row["itemLabel.value"]):
          ownerOf.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P47"):
        shareBorder.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P206"):
        bodyOfWater.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P37"):
        if(not location in row["itemLabel.value"]):
          languages.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P421"):
        timeZone.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P463"):
        memberOf.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P793"):
        if(not location in row["itemLabel.value"]):
          significantEvents.append(row["itemLabel.value"])

  #getting more than one result with one query, pick one for output
  if(len(ownerOf) > 0):
    listHintsLocation.append("The searched location is owner of " + ownerOf[0])
  if(len(shareBorder) > 0):
    listHintsLocation.append("The searched location shares border with " + shareBorder[0])
  if(len(bodyOfWater) > 0):
    listHintsLocation.append("The next body of water of searched location is " + bodyOfWater[0])
  if(len(memberOf) > 0):
    listBacklinks = getBacklinks(memberOf)
    dictPageViews = getPageViewsList(listBacklinks)
    listHintsLocation.append("Searched location is member of " + (max(dictPageViews, key=dictPageViews.get)))
  if(len(significantEvents) > 0):
    listBacklinks = getBacklinks(significantEvents)
    dictPageViews = getPageViewsList(listBacklinks)
    listHintsLocation.append("A significant event happened in this location was " + (max(dictPageViews, key=dictPageViews.get)))

  #if getting more than one language concatenate all
  commaCounter = len(languages)-1
  stringLanguages = ""
  for language in languages:
    if(commaCounter > 0):
      stringLanguages += language + ", "
      commaCounter -= 1
    else:
      stringLanguages += language
  if(stringLanguages != ""):
    listHintsLocation.append("Spoken language(s) in searched location is/are " + stringLanguages)
  if(len(timeZone) > 0):
    listHintsLocation.append("The searched location is in time Zone " + timeZone[0])

  
  return listHintsLocation


In [ ]:
def writeLocationHintsInFile(listHintsLocation):
  randomHintsList = random.sample(listHintsLocation, len(listHintsLocation))
  
  for item in randomHintsList:
    with open("hintsLocation.txt", 'a') as writefile:
      writefile.write(item + "\n")
      writefile.close()

##Hints if answer is a Person

In [ ]:
#Query with all properties to get data

def allHintsPerson(entity):
  sparql.setQuery('''
  SELECT ?itemLabel ?property
  WHERE
  {  
    VALUES ?property {wdt:P21 wdt:P27 wdt:P569 wdt:P19 wdt:P1971 wdt:P106 wdt:P1340 wdt:P1884 wdt:P2048 wdt:P39 wdt:P69 wdt:P512 wdt:P102 wdt:P3602 wdt:P800 wdt:P166 wdt:P3373 wdt:P1412 wdt:P413 wdt:P118 wdt:P54}
    wd:'''+entity+''' ?property ?item.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  hintsPerson = sparql.query().convert()

  hintsPerson_df = pd.json_normalize(hintsPerson['results']['bindings'])
  
  return hintsPerson_df

In [ ]:
#create temolate based hints

def getFormatedHintsPerson(queryResults_df):

  listOccupation = ["Searched person was occuupied as "]
  listPositionHeld = ["Searched person held the position(s) "]
  listEducatedAt = ["Searched person was educated at "]
  listAcademicDegrees = ["Searched person has academic degrees "]
  listPoliticParty = ["Searched was/is member of politic party "]
  listCandidacyElection = ["Searched person was candidacy at election(s) "]
  listNotableWork = ["Searched person did notable work "]
  listAwardsReceived = ["Searched person received awards "]
  listLanguages = ["Searched person speaks language(s) "]
  listPlayedPositions = ["Searched person played position "]
  listLeague = ["Searched person played in leagues "]
  listTeams = ["Searched person played in teams "]
  listCitizenship = ["Searched person has following citizenships "]
  listSiblings = []
  listHair = []

  listHintsPerson = []

  for index, row in queryResults_df.iterrows():
    if(row["property.value"] == "http://www.wikidata.org/prop/direct/P21"):
      listHintsPerson.append("Gender of searched person is " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P569"):
      listHintsPerson.append("Searched person was born in " + row["itemLabel.value"][:4])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P19"):
      listHintsPerson.append("Searched person was born in " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1971"):
      listHintsPerson.append("Number of children of seared person is " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1340"):
      listHintsPerson.append("Searched person has " + row["itemLabel.value"] + " eyes")    
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P2048"):
      listHintsPerson.append("Searched persons height is " + row["itemLabel.value"])    

    
    #if get multiple values save all in list and pick one afterwards
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P27"):
      listCitizenship.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P106"):
      listOccupation.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P39"):
      listPositionHeld.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P69"):
      listEducatedAt.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P512"):
      listAcademicDegrees.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P102"):
      listPoliticParty.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P3602"):
      listCandidacyElection.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P800"):
      listNotableWork.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P166"):
      listAwardsReceived.append(row["itemLabel.value"])

    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1412"):
      listLanguages.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P413"):
      listPlayedPositions.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P118"):
      listLeague.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P54"):
      listTeams.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P3373"):
      listSiblings.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1884"):
      listHair.append(row["itemLabel.value"])

  #if too many results could be return check
  #if more thane 3 -> cut
  listResultsCut = [listPositionHeld, listAcademicDegrees, listPoliticParty, 
                 listCandidacyElection, listNotableWork, listLanguages, 
                 listPlayedPositions, listLeague, listTeams, listCitizenship, listAwardsReceived, listEducatedAt, listOccupation]
  for singleList in listResultsCut:
    cutResults(singleList, listHintsPerson)

  #just get one hair color
  if(len(listHair) > 0):
    listHintsPerson.append("The searched person has " + listHair[0])

  #get number of siblings
  if(len(listSiblings) > 0):
    listHintsPerson.append("The searched person has " + str(len(listSiblings)) + " siblings")  
  
  return listHintsPerson

In [ ]:
#if more results than 3 -> print 3 results + "and x others"

def cutResults(listResults, listHintsPerson):
  
  numResults = len(listResults)-1
  resultsLeft = numResults-3

  if (len(listResults) > 1):
    if(resultsLeft > 0):
      listHintsPerson.append(getConcatenationMultipleResults(listResults[0:4])+ " and " + str(resultsLeft) + " others")
    else:
      listHintsPerson.append(getConcatenationMultipleResults(listResults))

In [ ]:
#concatenate multiple results with comma

def getConcatenationMultipleResults(resultList):
  concatenation = ""
  commaCounter = len(resultList)-1
  for item in resultList:
    if(commaCounter == len(resultList)-1):
      concatenation += item
      commaCounter-=1
    elif(commaCounter > 0):
      concatenation += item + ", "
      commaCounter-=1
    elif(commaCounter == 0):
      concatenation += item
  
  return concatenation


In [ ]:
def writePersonHintsInFile(listHintsPerson):
  with open("hintsPerson.txt", 'a') as writefile:
    #writefile.write("\n\n---Hints for Person " + person + "---\n\n")
    randomHintsList = random.sample(listHintsPerson, len(listHintsPerson))
    for item in randomHintsList:
      writefile.write(item + "\n")
    writefile.close()

## Hints if answer is a Year

**Get all outlinks of wikipedia year page and pass to getBacklinks. Than look for return values on page.content and print line**

In [ ]:
import spacy
def hintsYear(year):

  page = wikipedia.page(year, auto_suggest=False)
  pageContent = page.content

  #get all links from year page
  dictPageLinks = getOutlinks2(year)
  #print(dictPageLinks)

  #get rid of unnesercery links
  filteredPageLinks = filterDict(dictPageLinks)

  
  #get backlinks of all links
  dictBacklink = getBacklinksDict(filteredPageLinks)
  #first200 = {k: filteredPageLinks[k] for k in list(filteredPageLinks)[:200]}
  #dictBacklink = getBacklinksDict(first200)

  #remove parantheses of links
  dictAdjusted = removeParanthesesDict(dictBacklink)

  #get all possible hint (all lines with a entity #backlinks > 1000)
  dictPossibleHints = findLineInTextEvent(dictAdjusted, year)

  with open('./resultsEvents.txt', 'a') as writefile:
    writefile.write("Question: " + questionYear + "("+ str(year) + ")---\n")

  #check if entity because of which line was found is subject
  #if it is not the subject -> drop
  dictFinalHints = getFinalHints(dictPossibleHints)

  #get rid of number of foundevents at end of line
  #need to put number at the and because it is dict and would be overwritten
  dictFinalHintsAdjusted = {}
  for key, val in dictFinalHints.items():
    dictFinalHintsAdjusted.update({key.split('///', 1)[0]: val})  

  #returns dict sorted after PageViews
  dictSortedPV = sortHintsNumberPVSubject(dictFinalHintsAdjusted)

  #get highest PageView
  maxPV = list(dictSortedPV.values())[0]

  #sort hints PageViews and Simscore combined
  sortHintsUtilityScore(questionYear, dictSortedPV, maxPV)

  listHumansNames = []

  for key, val in dictAdjusted.items():
    result = checkIfHuman(key)
    if(result != None):
      listHumansNames.append(result)

  #findLineInTextBirthDeath(listHumansNames, year)

**Function to find line of most important link in year content**

In [ ]:
import re

def findLineInTextEvent(entitiesEvent, year):

  page = wikipedia.page(year, auto_suggest=False)
  pageText = page.content
  splitText = []

  splitText = (re.split("== Events ==|== Births ==", pageText ))

  contentEvents = splitText[1]

  foundEvent = 0

  dictHintsEntity = {}

  for key, val in entitiesEvent.items():
    #if (foundEvent == 20):
      #break;

    for line in contentEvents.split("\n"):
      #if (foundEvent == 20):
        #break;

      if val in line:
        words = line.split()
        #counterEntitiesInLine = 0
        if(len(words) < 15):
          dictHintsEntity.update({line + "///" + str(foundEvent): val})
          #listHints.append(line)
          foundEvent += 1
          #break;
  return dictHintsEntity

In [ ]:
def findLineInTextBirthDeath(entitiesHuman, year):

  page = wikipedia.page(year, auto_suggest=False)
  pageText = page.content
  splitText = []

  splitText = (re.split("== Births ==|== Deaths ==", pageText ))

  print(splitText)

  contentBirths = splitText[1]
  contentDeaths = splitText[2]

  foundBirth = 0
  foundDeath = 0

  #open file and append results
  with open('./resultsBirthDeath.txt', 'a') as writefile:
    writefile.write("---Birth/Death in year " + str(year) + "---")

    for entity in entitiesHuman:
      #if (foundBirth == 3 and foundDeath == 3):
        #writefile.write("\n")
        #break;

      for line in contentBirths.split("\n"):
        #if (foundBirth == 3):
          #break;

        if entity in line:
          writefile.write("Found with entity " + entity + "\n")
          writefile.write("Birth: " + line + "\n")
          writefile.write("\n")
          foundBirth +=1
          break;

      for line in contentDeaths.split("\n"):
        #if (foundDeath == 3):
          #break;
        if entity in line:
          writefile.write("Found with entity " + entity + "\n")
          writefile.write("Death: " + line + "\n")
          writefile.write("\n")
          foundDeath += 1
          break;
    writefile.write("\n")
      

In [ ]:
def sortHintsNumberPVSubject(dictHints):

  with open('./resultsEvents.txt', 'a') as writefile: 

    dictFinalHintsPV = {}

    for key, val in dictHints.items():
      dictFinalHintsPV.update({key: getPageViews(val)})

    dictFinalHintsSortedPV = dict(sorted(dictFinalHintsPV.items(), key=lambda item: item[1], reverse=True))
    """writefile.write("\n--Final Hints sorted #PV---\n")
    for key, val in dictFinalHintsSortedPV.items():
      writefile.write(key + "(" + str(val) + " PageViews)\n")"""

  return dictFinalHintsSortedPV



In [ ]:


def sortHintsUtilityScore(question, dictFinalHintsSortedPV, maxPV):
  model = SentenceTransformer('bert-base-nli-mean-tokens')

  question_embedding = model.encode(question)

  alpha = 0.5

  #sentence_embeddings = model.encode(hints)

  dictHintsBERT = {}

  for key, val in dictFinalHintsSortedPV.items():
    sentence_embedding = model.encode(key)
    simScore = cosine_similarity([question_embedding], [sentence_embedding]).item()
    impScore = alpha * (val/maxPV) + (1-alpha) * simScore
    dictHintsBERT.update({key: impScore})

  with open('./resultsEvents.txt', 'a') as writefile:

    dictHintsBERT = dict(sorted(dictHintsBERT.items(), key=lambda item: item[1], reverse=True))
    writefile.write("\n--Final Hints sorted Utility Score---\n")
    for key, val in dictHintsBERT.items():
      writefile.write(key + "(" + str(val) + " Utility Score)\n")

  return dictHintsBERT
 

In [ ]:
def get_subject_phrase(doc):
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [ ]:
def getFinalHints(dictPossibleHints):

  #nlp = spacy.load("en")
  nlp = spacy.load("en_core_web_sm")
  dictFinalHintsSubject = {}
  
  for key, val in dictPossibleHints.items():
    doc = nlp(key)
    subject = get_subject_phrase(doc)
    if (val in str(subject)):
      dictFinalHintsSubject.update({key: val})
      #listFinalHints.append(key)

  return dictFinalHintsSubject

In [ ]:

def getOutlinks2(year):

  URL = 'https://en.wikipedia.org/wiki/' + str(year)

  # Fetch all the HTML source from the url
  response = requests.get(URL)


  soup = bs4.BeautifulSoup(response.text, 'html.parser')

  listTextOfLinks = []
  listLinks = []

  dictLinkName = {}


  for link in soup.find_all("a"):
    listTextOfLinks.append(link.get_text())
    dictLinkName.update({str(link.get("href")).split('/')[-1].replace('_', ' '): link.get_text()})

  #newDict = filterDict(dictLinkName)

  #print(len(dictLinkName))
  #print(len(newDict))
  #print(newDict)

  return dictLinkName

In [ ]:
def filterDict(dictNameLink):

  patternYear = r"[1-3][0-9]{3}"
  patternCentury = r"[1-2][0-9]\w+.century"
  patternAd = r"AD.\d\d\d\d"
  patternMonth = r"[a-zA-Z]+\s\d+"
  patternMillennium = r".+millennium"
  patternHashtag = r"#"
  patternCalendar = r"calendar"
  patternCategory = r"category"
  patternList = r"list"
  patternTemplate = r"template"
  patternWikipedia = r"wikipedia"
  patternIndex = r"index"
  patternHtml = r"html"
  patternPercent = r"%"

  newDict = {}

  for key, val in dictNameLink.items():
    if(not re.match(patternYear, key) and
       not re.match(patternCentury, key) and 
       not re.match(patternAd, key) and 
       not re.match(patternMonth, key) and
       not re.search(patternHashtag, key) and
       not re.search(patternCalendar, key, re.IGNORECASE) and
       not re.search(patternCategory, key, re.IGNORECASE) and
       not re.search(patternList, key, re.IGNORECASE) and
       not re.search(patternTemplate, key, re.IGNORECASE) and
       not re.search(patternWikipedia, key, re.IGNORECASE) and
       not re.search(patternIndex, key, re.IGNORECASE) and
       not re.search(patternHtml, key, re.IGNORECASE) and
       not re.search(patternPercent, key) and
       not re.match(patternMillennium, key) and
       not val == "" and not val == " "):
      

      newDict.update({key: val})


  print(newDict)

  return newDict


## Call Functions


**Call hint function for all questions where answer is Location**

In [ ]:
for index, row in location_df.iterrows():
  location = row["Answer"]
  questionLocation = row["Question"]
  with open("hintsLocation.txt", 'a') as writefile:
    writefile.write("\n\n" + questionLocation + " (" + location + ")\n\n")
    writefile.close()


  #get Wikidata ID for SPARQL-Query
  locationQitem = getQitemOfName(location)

  #execute SPARQL-Query
  queryResults_df = allHintsLocation(locationQitem)

  #getFinalHints
  listHintsLocation = getFormatedHintsLocation(queryResults_df)    

  #write hints in file in random order
  writeLocationHintsInFile(listHintsLocation)

**Call hint function for all questions where answer is Person**



In [ ]:

for index, row in person_df.iterrows():
  person = row["Answer"]
  questionPerson = row["Question"]

  with open("hintsPerson.txt", 'a') as writefile:
    writefile.write("\n\n" + questionPerson + " ("+ person +")\n\n")
    writefile.close()
  
  #get Wikipedia ID for SPARQL-Query
  personQitem = getQitemOfName(person)

  #execute SPARQL-Query
  queryResults_df = allHintsPerson(personQitem)
  
  #get Final Hints
  listHintsPerson = getFormatedHintsPerson(queryResults_df)

  #write hints in file in random order
  writePersonHintsInFile(listHintsPerson)


**Call hint function for all question where answer is Year**

In [ ]:
for index, row in year_df.iterrows():
  year = row["Answer"]
  questionYear = row["Question"]
  hintsYear(row["Answer"])

# Pageview Test Functions


In [ ]:
import time
import requests

from selenium import webdriver
from bs4 import BeautifulSoup

def get_table_info(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    options = webdriver.FirefoxOptions()
    #options.headless = True
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(3) # Wait for the page to load completely
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])
    return (headers, data)

def get_table_info_requests(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    headers = []
    data = []

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])

    return (headers, data)

def get_links_in_section(wikipedia_url, section_heading):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link on that section of the page.
    """
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('span', {'id': section_heading})
    if section is None:
        return None
    section_content = section.parent.find_next_sibling('ul')
    if section_content is None:
        return None
    links = []
    for item in section_content.find_all('li'):
        link = item.find('a')
        if link is not None and link.has_attr('href') and link['href'].startswith('/wiki/'):
            #title = link.get('title', '')
            title = link.get('title')
            description = item.text.strip()
            url = f"https://en.wikipedia.org{link['href']}"
            links.append({'title': title, 'description': description, 'url': url})
    #print(links)
    return links

def get_links_in_section_with_sublinks(wikipedia_url, section_title):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link  with its sublinks as well.
    """
    response = requests.get(wikipedia_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        section_heading = soup.find('span', {'id': section_title})
        if section_heading is None:
            return None
        section = section_heading.parent
        section_content = section.find_next_sibling('ul')
        if section_content is None:
            return None
        links = []
        for item in section_content.find_all('li'):
            link = item.find('a')
            if link is not None:
                link_title = link.get('title')
                link_url = 'https://en.wikipedia.org' + link.get('href')
                link_description = item.text.replace(link_title, '').strip()
                links.append({'title': link_title, 'url': link_url, 'description': link_description})
                for sublink in item.find_all('a', href=True, recursive=False):
                    sublink_title = sublink.get('title')
                    sublink_url = 'https://en.wikipedia.org' + sublink.get('href')
                    sublink_description = sublink.parent.text.replace(sublink_title, '').replace(link_title, '').strip()
                    links.append({'title': sublink_title, 'url': sublink_url, 'description': sublink_description})
        return links
    else:
        return None

def get_events_links(title):
    """
    Gets all the links in the "Events" section of a Wikipedia page.
    Returns a list of strings containing the URLs of each event.
    """
    url = f"https://en.wikipedia.org/w/api.php?action=parse&format=json&page={title}&prop=text"
    response = requests.get(url)
    data = response.json()

    soup = BeautifulSoup(data['parse']['text']['*'], 'html.parser')
    events_section = soup.find('span', {'class': 'mw-headline', 'id': 'Events'})
    if events_section:
        events_list = events_section.find_next('ul')
        if events_list:
            events = events_list.find_all('a')
            event_links = [f"https://en.wikipedia.org{event['href']}" for event in events]
            return event_links
    return None

def create_url_from_title(title):
    """
    Given a list of dictionaries containing href, title, and description for each entry, extracts the title from each link and constructs a new URL using the title.
    """
    urls = []
    if title != None: 
      # Replace spaces with underscores in the title
      title = title.replace(' ', '_')

      # Construct a new URL using the title
      url = 'https://pageviews.wmcloud.org/redirectviews/?project=en.wikipedia.org&platform=all-access&agent=user&range=all-time&sort=views&direction=1&view=list&page=' + title
    return url

def extract_visitorNumber_from_link(link):
    """
    extract the first entry of the table
    """
    headers, data = get_table_info(link)

    try:
      if data[1]: 
        return data[1][2]
      else:
        return 0
    except IndexError:
      print("Index out of range")

def get_visitor_numbers(pageviews_url):
    """
    returns visitor numbers of the page
    """
    response = requests.get(pageviews_url)
    if response.status_code == 200:
        data = response.json()['items']
        visitors = {}
        for item in data:
            visitors[item['timestamp']] = item['views']
        return visitors
    else:
        return None

In [ ]:
#TEST

#Seitenaufrufe
#url = 'https://pageviews.toolforge.org/?project=en.wikipedia.org&pages=Formula1|Michael_Schumacher'
#headers, data = get_table_info(url)
#print(headers)
#print(data)

#get all wiki links on a page -> discard the one we dont need -> analyse the ones that could be usefull -> sort them after the backlinks
#print(get_events_links(2004))

# Search the most popular births and deaths in that year and then order them 
url1 = 'https://en.wikipedia.org/wiki/Category:1994_births'
url2 = 'https://en.wikipedia.org/wiki/Category:1994_deaths'
url3 = 'https://en.wikipedia.org/wiki/1994#Events'

#links_events_section = get_links_in_section_with_sublinks(url3, 'Events')
#print(len(links_events_section))
#print(links_events_section)
#print(get_births(1994))

#visitor number test
link = 'https://pageviews.wmcloud.org/redirectviews/?project=en.wikipedia.org&platform=all-access&agent=user&range=all-time&sort=views&direction=1&view=list&page=North_American_Free_Trade_Agreement'
#print(extract_visitorNumber_from_link(link))

In [ ]:
#When did Michael Schumacher win his first F1 World Drivers Title? -> 1994
#First 1994 is searched on wikipedia, then all the links from the event sections of that page are filtered and written into a dict together with their pageviews.
#Now order these lists such that the entry with most pageviews is on top
#THIS SHOULD WORK

year=2004
url = 'https://en.wikipedia.org/wiki/2004'

links_events_section = get_links_in_section_with_sublinks(url, 'Events')
print(len(links_events_section))

for link in links_events_section:
    t_title =  link.get('title') 
    t_url = create_url_from_title(t_title)
    link["pageview_url"] = t_url
    link["pageviews"] = extract_visitorNumber_from_link(t_url)

#sort the list links_event_section after the pageviews such that the entity with the highest pageviews is first
sorted_list = sorted(links_events_section, key=lambda x: int(x['pageviews'].replace(',', '')), reverse=True)

In [ ]:
import pprint
pageview_test_generated_hints_for_years = sorted_list

pprint.pprint(pageview_test_generated_hints_for_years, indent=4)

# Clean GitHub directory




In [ ]:
%cd /content/
%rm -r optimizationOfHintGeneration
%cd /content/
%rm hintsLocation.txt
%rm hintsPerson.txt
%rm resultsEvents.txt